In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import openpyxl
import warnings
warnings.filterwarnings("ignore")

# Load Variables

* passengers includes the number of passengers in the desired rider class
* fare_track_miles includes the fare per track mile
* auto_tt_minpmile includes the interpolated auto travel times
* bus_tt_minpmile includes the bus travel times per mile
* households_half_mile includes the proportional number of households within each walkshed
* am_parking_users includes the number of parking users per OD pair
* num_bus_lines contains the number of bus lines and stops
* jobs_half_mile contains the proportional number of jobs within each walkshed
* trains_per_hour contains the number of trains in each time period (only AM will be used)
* terminal_station contains a dummy variable of whether or not a station is at the end of a line (1) or not (0)
* dist_to_core contains the distance from a station to the Metro Center station in miles
* parking_capacity contains the number of parking spots available per station
* mstns contains the station names and ID's used for merging the variables together

## .5 Level and OD vars

In [2]:
#passengers = pd.read_excel("../Data Preprocessing/Ridership Data/output/full_fare_ridership.xlsx")
fare_track_miles      = pd.read_excel("../Data Preprocessing/Fare per Track Mile/output/railOD_peak_fare_per_mile.xlsx")
fare_track_miles_all  = pd.read_excel("../Data/railOD_TravelTimesAndFares.xlsx")
auto_tt_minpmile      = pd.read_csv("../Data Preprocessing/Interpolated Auto Travel Times/output/am_interpolated_auto_times.csv", index_col =  0)
bus_tt_minpmile       = pd.read_excel("../Data Preprocessing/Bus Travel Time/output/busttpermile_ML.xlsx") 
households_half_mile  = pd.read_excel("../Data Preprocessing/Proportion of Households/output/walkshed_proportional_households_stations_p5.xlsx") 
am_parking_users      = pd.read_csv("../Data Preprocessing/Parking Users/output/am_parking_updated.csv")
num_bus_lines         = pd.read_csv("../Data Preprocessing/Bus Lines and Stops/output/bus_line_stop_p5.csv")
jobs_half_mile        = pd.read_excel("../Data Preprocessing/Proportion of Jobs/output/proportional_jobs_stations_p5.xlsx")
trains_per_hour       = pd.read_excel("../Data/avgTrainThroughput_byPeriod_PeakOnly.xlsx") 
terminal_station      = pd.read_csv("../Data/metro_ternimal_dummy.csv")
track_miles           = pd.read_excel("../Data/railOD_trackMiles_spring2022.xlsx", header=1, index_col=0)
dist_to_core          = pd.read_excel("../Data Preprocessing/Distance to the Core/output/stations.xlsx")
parking_capacity      = pd.read_excel("../Data/parkingCapacityByStation.xlsx")
pm_parking_users      = pd.read_csv("../Data Preprocessing/Parking Users/output/pm_parking_updated.csv")
pm_auto_tt            = pd.read_csv("../Data Preprocessing/Interpolated Auto Travel Times/output/pm_interpolated_auto_times.csv", index_col = 0)
off_parking_users     = pd.read_csv("../Data Preprocessing/Parking Users/output/off_parking_updated.csv")
off_fare_track_miles  = pd.read_excel("../Data Preprocessing/Fare per Track Mile/output/railOD_off_peak_fare_per_mile.xlsx")
nine_five_jobs        = pd.read_excel("../Data Preprocessing/Proportion of 9-5 Jobs/output/proportional_ninetofive_workers_stations_p5.xlsx", index_col=0)
school_jobs           = pd.read_excel("../Data Preprocessing/Proportion of Education Jobs/output/proportion_education_jobs_stations_p5.xlsx")
nite_wkend_jobs       = pd.read_excel("../Data Preprocessing/Proportion of Night and Weekend Jobs/output/Proportional_night_weekend_jobs_ML_p5.xlsx")
median_income         = pd.read_excel("../Data Preprocessing/Median Household Income/output/proportional_walkshed_household_income_updated_p5.xlsx", index_col=0)
off_auto_tt           = pd.read_csv("../Data Preprocessing/Interpolated Auto Travel Times/output/off_interpolated_auto_times.csv", index_col = 0)
mstns                 = pd.read_excel("../Data/mstn_id_to_stn_name.xlsx")

# updated variables. 
all_trains_per_hour   = pd.read_csv("../Data Preprocessing/Averaged Trains Per Hour/output/trains_per_hour_all_periods.csv", index_col=0)
hh_incomebelowpoverty = pd.read_csv("../Data Preprocessing/HH_IncomeBelowPoverty/output/HH_Income_povertylevel_p5.csv")
hotels                = pd.read_csv("../Data Preprocessing/Hotels Restaurants Entertainment/output/hotels_by_station_p5.csv").drop('Unnamed: 0', axis=1)
entertainment         = pd.read_csv("../Data Preprocessing/Hotels Restaurants Entertainment/output/entertainment_by_station_p5.csv").drop('Unnamed: 0', axis=1)
restaurant            = pd.read_csv("../Data Preprocessing/Hotels Restaurants Entertainment/output/restaurants_by_station_p5.csv").drop('Unnamed: 0', axis=1)
inter_city            = pd.read_csv("../Data Preprocessing/Intercity hubs/output/inter_cityhub_p5.csv")
intersection_density  = pd.read_csv("../Data Preprocessing/Intersection Density/output/proportion_density_walkstation_p5.csv")
job_access_by_transit = pd.read_csv("../Data Preprocessing/Job Accessibility 45mins by transit and Auto/output/job_auto_and_transit_accessibity45mins_p5.csv")
convenience_store     = pd.read_csv("../Data Preprocessing/number of convenience stores/output/convenience_by_station_p5.csv").drop('Unnamed: 0', axis=1)
highschool_enrollment = pd.read_csv("../Data Preprocessing/Number of education institutions/output/Enrollment_highschool_p5.csv")
college_enrollment    = pd.read_csv("../Data Preprocessing/Number of education institutions/output/Enrollment_colleges_p5.csv")
highschool_hh         = pd.read_csv("../Data Preprocessing/Number of Student Population/output/High_school_student_population_Enrollment_p5.csv")
college_hh            = pd.read_csv("../Data Preprocessing/Number of Student Population/output/grad_student_population_Enrollment_p5.csv")
hh_0_car              = pd.read_csv("../Data Preprocessing/Number of Households with no car or one car/output/victor-no of household w no car_p5.csv")
hh_1_car              = pd.read_csv("../Data Preprocessing/Number of Households with no car or one car/output/victor-no of household w 1 car_p5.csv")
hh_morethan1_car      = pd.read_csv("../Data Preprocessing/Number of Households with no car or one car/output/victor-no of household w more than 1cars_p5.csv")
public_admin_jobs     = pd.read_csv("../Data Preprocessing/Public Admin Jobs/output/proportional_publicadmin_workers_stations_p5.csv")
senior_housing_afford = pd.read_csv("../Data Preprocessing/seniorhousing_affordability/output/senior_housing_facilitiesDMV_p5.csv").drop('Unnamed: 0', axis=1)
bike_tt               = pd.read_csv("../Data Preprocessing/bike travel time/output/bike_tt.csv").drop('count', axis=1)
bikeshare_cap         = pd.read_csv("../Data Preprocessing/bikeshare capacity/output/bikeshare_capacity_p5.csv")
bus_comp_index        = pd.read_csv("../Data Preprocessing/bus competitiveness/bus_competativeness_index.csv")
google_auto_times     = pd.read_csv("../Data Preprocessing/Google Travel Times/google_driving_summaries.csv")
ctpp_jobs             = pd.read_csv("../Data Preprocessing/CTPP Jobs Data/output/CTPP_jobs_p5.csv")
metro_swipe_tt        = pd.read_excel("../Data Preprocessing/Calculated Metro Travel Times/output/metro_swipecard_tt.xlsx", index_col=0)
sop                   = pd.read_csv("../Data Preprocessing/SoP Data/output/SoP_BikeLanes_p5.csv", index_col = 0)


# All riders
#all_riders = pd.read_csv("../Data Preprocessing/All Ridership/output/WMATA_entrances_exits_all_riders.csv").drop('Unnamed: 0', axis=1)

#### Merge the recent variables in a single df

* Sort the station names alphabetically
* Make the mstn file have columns of alternatively formatted station names to help with merging

In [3]:
mstns.sort_values(by = 'PRIMARY_NAME_FY23',inplace=True)

walkshed_filename = ['ADDISON ROAD-SEAT PLEASANT','ANACOSTIA','ARCHIVES-NAVY MEMORIAL-PENN QUARTER','ARLINGTON CEMETERY','Ashburn','BALLSTON-MU',
                    'BENNING ROAD','BETHESDA','BRADDOCK ROAD','BRANCH AVE','BROOKLAND-CUA','CAPITOL HEIGHTS','CAPITOL SOUTH','CHEVERLY',
                    'CLARENDON','CLEVELAND PARK','COLLEGE PARK-U OF MD','COLUMBIA HEIGHTS','CONGRESS HEIGHTS','COURT HOUSE','CRYSTAL CITY',
                    'DEANWOOD','LARGO TOWN CENTER','Washington Dulles International Airport', 'DUNN LORING-MERRIFIELD','DUPONT CIRCLE','EAST FALLS CHURCH','EASTERN MARKET',
                    'EISENHOWER AVENUE','FARRAGUT NORTH', 'FARRAGUT WEST','FEDERAL CENTER SW','FEDERAL TRIANGLE','FOGGY BOTTOM-GWU','FOREST GLEN','FORT TOTTEN','FRANCONIA-SPRINGFIELD',
                    'FRIENDSHIP HEIGHTS','GALLERY PL-CHINATOWN','GEORGIA AVE-PETWORTH','GLENMONT','GREENBELT','GREENSBORO','GROSVENOR-STRATHMORE',
                    'Herndon','HUNTINGTON',"PRINCE GEORGE'S PLAZA",'Innovation Center','JUDICIARY SQUARE','KING ST-OLD TOWN',"L'ENFANT PLAZA",'LANDOVER',
                    'Loudoun Gateway','MCLEAN','MCPHERSON SQUARE','MEDICAL CENTER','METRO CENTER','MINNESOTA AVE','MORGAN BOULEVARD',
                    'MT VERNON SQ 7TH ST-CONVENTION CENTER','NAVY YARD-BALLPARK','NAYLOR ROAD','NEW CARROLLTON','NOMA-GALLAUDET','WHITE FLINT','PENTAGON',
                    'PENTAGON CITY','POTOMAC AVE', "",'Reston Town Center','RHODE ISLAND AVE-BRENTWOOD','ROCKVILLE','RONALD REAGAN WASHINGTON NATIONAL AIRPORT',
                    'ROSSLYN','SHADY GROVE','SHAW-HOWARD U','SILVER SPRING','SMITHSONIAN','SOUTHERN AVENUE','SPRING HILL',
                    'STADIUM-ARMORY','SUITLAND','TAKOMA','TENLEYTOWN-AU','TWINBROOK','TYSONS CORNER','U STREET/AFRICAN-AMER CIVIL WAR MEMORIAL/CARDOZO',
                    'UNION STATION','VAN DORN STREET','VAN NESS-UDC','VIENNA/FAIRFAX-GMU','VIRGINIA SQUARE-GMU','WATERFRONT','WEST FALLS CHURCH-VT/UVA',
                    'WEST HYATTSVILLE','WHEATON','WIEHLE-RESTON EAST','WOODLEY PARK-ZOO/ADAMS MORGAN',]


WMATA_filename =  ['Addison Road', 'Anacostia', 'Archives-Navy Memorial', 'Arlington Cemetery', "", 'Ballston', 'Benning Road', 'Bethesda', 'Braddock Road', 
                   'Branch Avenue', 'Brookland', 'Capitol Heights', 'Capitol South', 'Cheverly', 'Clarendon', 'Cleveland Park', 'College Park-U of MD', 'Columbia Heights',
                   'Congress Heights', 'Court House', 'Crystal City', 'Deanwood','Largo Town Center', "", 'Dunn Loring', 'Dupont Circle', 'East Falls Church', 
                   'Eastern Market', 'Eisenhower Avenue', 'Farragut North', 'Farragut West', 'Federal Center SW', 'Federal Triangle', 'Foggy Bottom', 'Forest Glen', 
                   'Fort Totten', 'Franconia-Springfield', 'Friendship Heights', 'Gallery Place-Chinatown', 'Georgia Avenue-Petworth', 'Glenmont', 'Greenbelt', 
                   'Greensboro', 'Grosvenor', "", 'Huntington',"Prince George's Plaza", "", 'Judiciary Square', 'King Street', "L'Enfant Plaza", 'Landover', "", 
                   'McLean', 'McPherson Square', 'Medical Center', 'Metro Center', 'Minnesota Avenue', 'Morgan Blvd.', 'Mt. Vernon Square-UDC', 'Navy Yard', 
                   'Naylor Road', 'New Carrollton', 'New York Ave','White Flint', 'Pentagon', 'Pentagon City', 'Potomac Avenue', "", "",'Rhode Island Avenue',
                   'Rockville', 'Reagan Washington National Airport', 'Rosslyn', 'Shady Grove', 'Shaw-Howard University', 'Silver Spring', 'Smithsonian', 'Southern Avenue', 
                   'Spring Hill', 'Stadium-Armory', 'Suitland', 'Takoma', 'Tenleytown-AU', 'Twinbrook', 'Tysons Corner', 'U Street-Cardozo', 'Union Station', 'Van Dorn Street', 
                   'Van Ness-UDC', 'Vienna', 'Virginia Square-GMU', 'Waterfront', 'West Falls Church', 'West Hyattsville', 'Wheaton', 'Wiehle', 'Woodley Park-Zoo']


track_mile_names = ['Addison Road', 'Anacostia', 'Archives', 'Arlington Cemetery', "", 'Ballston-MU', 'Benning Road', 'Bethesda', 'Braddock Road', 'Branch Ave', 'Brookland-CUA', 
                    'Capitol Heights', 'Capitol South', 'Cheverly', 'Clarendon', 'Cleveland Park', 'College Park-U of Md', 'Columbia Heights', 'Congress Heights', 'Court House',
                    'Crystal City', 'Deanwood', 'Largo Town Center', "", 'Dunn Loring', 'Dupont Circle', 'East Falls Church', 'Eastern Market', 'Eisenhower Ave', 'Farragut North',
                    'Farragut West', 'Federal Center SW', 'Federal Triangle', 'Foggy Bottom-GWU', 'Forest Glen', 'Fort Totten', 'Franconia-Springfield', 'Friendship Heights', 
                    'Gallery Place', 'Georgia Ave-Petworth', 'Glenmont', 'Greenbelt', 'Greensboro', 'Grosvenor-Strathmore', "",'Huntington',"Prince George's Plaza", "", 'Judiciary Square', 
                    'King St-Old Town', "L'Enfant Plaza", 'Landover',"", 'McLean', 'McPherson Square', 'Medical Center', 'Metro Center', 'Minnesota Ave', 'Morgan Boulevard', 'Mt Vernon Sq', 
                    'Navy Yard-Ballpark', 'Naylor Road', 'New Carrollton', 'NoMa-Gallaudet U','White Flint', 'Pentagon', 'Pentagon City', 'Potomac Ave', "","", 'Rhode Island Ave',
                    'Rockville', 'Ronald Reagan Washington National Airport', 'Rosslyn', 'Shady Grove', 'Shaw-Howard Univ', 'Silver Spring', 'Smithsonian', 'Southern Ave', 'Spring Hill', 
                    'Stadium-Armory', 'Suitland', 'Takoma', 'Tenleytown-AU', 'Twinbrook', 'Tysons Corner', 'U Street', 'Union Station', 'Van Dorn Street', 'Van Ness-UDC', 'Vienna', 
                    'Virginia Square-GMU', 'Waterfront', 'West Falls Church', 'West Hyattsville', 'Wheaton',  'Wiehle', 'Woodley Park']

auto_filename = ['ADDISON ROAD-SEAT PLEASANT', 'ANACOSTIA', 'ARCHIVES-NAVY MEMORIAL-PENN QUARTER', 'ARLINGTON CEMETERY', 'ASHBURN', 'BALLSTON-MU', 'BENNING ROAD', 'BETHESDA', 
                 'BRADDOCK ROAD', 'BRANCH AVE', 'BROOKLAND-CUA', 'CAPITOL HEIGHTS', 'CAPITOL SOUTH', 'CHEVERLY', 'CLARENDON', 'CLEVELAND PARK', 'COLLEGE PARK-U OF MD', 'COLUMBIA HEIGHTS', 
                 'CONGRESS HEIGHTS', 'COURT HOUSE', 'CRYSTAL CITY', 'DEANWOOD', 'DOWNTOWN LARGO', 'WASHINGTON DULLES INTERNATIONAL AIRPORT', 'DUNN LORING-MERRIFIELD', 'DUPONT CIRCLE', 
                 'EAST FALLS CHURCH', 'EASTERN MARKET', 'EISENHOWER AVENUE', 'FARRAGUT NORTH', 'FARRAGUT WEST', 'FEDERAL CENTER SW', 'FEDERAL TRIANGLE', 'FOGGY BOTTOM-GWU', 'FOREST GLEN', 
                 'FORT TOTTEN', 'FRANCONIA-SPRINGFIELD', 'FRIENDSHIP HEIGHTS', 'GALLERY PLACE-CHINATOWN', 'GEORGIA AVENUE-PETWORTH', 'GLENMONT', 'GREENBELT', 'GREENSBORO', 'GROSVENOR-STRATHMORE', 
                 'HERNDON', 'HUNTINGTON', 'HYATTSVILLE CROSSING', 'INNOVATION CENTER', 'JUDICIARY SQUARE', 'KING STREET-OLD TOWN', "L'ENFANT PLAZA", 'LANDOVER', 'LOUDOUN GATEWAY', 'MCLEAN',
                 'MCPHERSON SQUARE', 'MEDICAL CENTER', 'METRO CENTER', 'MINNESOTA AVENUE', 'MORGAN BOULEVARD', 'MT VERNON SQ 7TH ST-CONVENTION CENTER', 'NAVY YARD-BALLPARK', 'NAYLOR ROAD',
                 'NEW CARROLLTON', 'NOMA-GALLAUDET U', 'NORTH BETHESDA', 'PENTAGON', 'PENTAGON CITY', 'POTOMAC AVE', "",'RESTON', 'RHODE ISLAND AVE-BRENTWOOD', 'ROCKVILLE',
                 'RONALD REAGAN WASHINGTON NATIONAL AIRPORT', 'ROSSLYN', 'SHADY GROVE', 'SHAW-HOWARD UNIVERSITY', 'SILVER SPRING', 'SMITHSONIAN', 'SOUTHERN AVENUE', 'SPRING HILL',
                 'STADIUM-ARMORY', 'SUITLAND', 'TAKOMA', 'TENLEYTOWN-AU', 'TWINBROOK', 'TYSONS', 'U STREET/AFRICAN-AMER CIVIL WAR MEMORIAL/CARDOZO', 'UNION STATION', 'VAN DORN STREET', 
                 'VAN NESS-UDC', 'VIENNA/FAIRFAX-GMU', 'VIRGINIA SQUARE-GMU', 'WATERFRONT', 'WEST FALLS CHURCH-VT/UVA', 'WEST HYATTSVILLE', 'WHEATON', 'WIEHLE-RESTON EAST',
                 'WOODLEY PARK-ZOO/ADAMS MORGAN']



mstns['walkshed_filename'] = walkshed_filename

mstns['WMATA_filename'] =  WMATA_filename

mstns['track_miles_names'] = track_mile_names

mstns['auto_filename'] = auto_filename

# OD Variables

## AM Passenger Miles

* Filter for the AM trip passengers
* Format track_miles into OD pairs
* Give station names in track_miles proper MSTN IDs
* Create an od_pair key column in both the passengers and track miles dataframes
* merge track miles to passengers in the am, multiply the track miles by passengers, consolidate the columns, rename to passenger column

In [4]:
track_miles = track_miles.stack().rename_axis(('O',"D")).reset_index(name='track_miles')

In [5]:
track_miles2 = mstns.merge(track_miles, left_on ='track_miles_names' , right_on= 'O')
track_miles2 = mstns.merge(track_miles2, left_on ='track_miles_names' , right_on='D' )

In [6]:
track_miles2['pairs'] = track_miles2.apply(lambda x: str(x['ID_y'])+str(0)+str(x['ID_x']), axis=1)


In [7]:
allvars_dataframe_new = track_miles2
allvars_dataframe_new.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,track_miles_names_y,auto_filename_y,O,D,track_miles,pairs
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,0.10,MSTN_0620MSTN_062
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,Anacostia,ANACOSTIA,Anacostia,Addison Road,10.35,MSTN_0010MSTN_062
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives,Addison Road,8.31,MSTN_0020MSTN_062
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,Addison Road,11.39,MSTN_0650MSTN_062
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,Ballston-MU,BALLSTON-MU,Ballston-MU,Addison Road,14.27,MSTN_0680MSTN_062


In [8]:
allvars_dataframe_new.shape

(8281, 16)

## Fare

In [9]:
fare_track_miles_all['pairs'] = fare_track_miles_all.apply(lambda x: str(x['O_MSTN_ID'])+str(0)+str(x['D_MSTN_ID']), axis=1)
fare_track_miles_all.head()

,O_MSTN_ID,D_MSTN_ID,O_PRIMARY_NAME,D_PRIMARY_NAME,COMP_MILE,PEAK_FARE,OFF_PEAK_FARE,SD_FARE,TRAVEL_TIME,pairs
0,MSTN_001,MSTN_001,Anacostia,Anacostia,0.05,2.25,2.0,1.10,0,MSTN_0010MSTN_001
1,MSTN_001,MSTN_002,Anacostia,Archives,2.83,2.25,2.0,1.10,9,MSTN_0010MSTN_002
2,MSTN_001,MSTN_003,Anacostia,Benning Road,5.76,3.15,2.6,1.55,26,MSTN_0010MSTN_003
3,MSTN_001,MSTN_004,Anacostia,Brookland-CUA,5.98,3.20,2.6,1.60,27,MSTN_0010MSTN_004
4,MSTN_001,MSTN_005,Anacostia,Capitol South,2.55,2.25,2.0,1.10,16,MSTN_0010MSTN_005


In [10]:
allvars_dataframe_new = allvars_dataframe_new.merge(fare_track_miles_all[['COMP_MILE', 'PEAK_FARE', 'OFF_PEAK_FARE', 'SD_FARE', 'TRAVEL_TIME',
       'pairs']], on = 'pairs', how = 'left')
allvars_dataframe_new.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,auto_filename_y,O,D,track_miles,pairs,COMP_MILE,PEAK_FARE,OFF_PEAK_FARE,SD_FARE,TRAVEL_TIME
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,0.10,MSTN_0620MSTN_062,0.05,2.25,2.00,1.10,0
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,...,ANACOSTIA,Anacostia,Addison Road,10.35,MSTN_0010MSTN_062,8.03,3.80,3.15,1.90,31
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,...,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives,Addison Road,8.31,MSTN_0020MSTN_062,7.61,3.70,3.10,1.85,26
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,...,ARLINGTON CEMETERY,Arlington Cemetery,Addison Road,11.39,MSTN_0650MSTN_062,10.26,4.45,3.65,2.20,35
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,...,BALLSTON-MU,Ballston-MU,Addison Road,14.27,MSTN_0680MSTN_062,13.03,5.25,3.85,2.60,41


In [11]:
allvars_dataframe_new.shape

(8281, 21)

## Peak Fare per Track Mile

* Subset only the station ID columns from fare_track_miles
* Create a unique OD pair column
* Merge into the main AM dataframe

In [12]:
fare_track_miles= fare_track_miles[['O_MSTN_ID', 'D_MSTN_ID', 'peak_fare_per_mile2']]

In [13]:
fare_track_miles['pairs'] = fare_track_miles.apply(lambda x: str(x['O_MSTN_ID'])+str(0)+str(x['D_MSTN_ID']), axis=1)

In [14]:
allvars_dataframe_new = allvars_dataframe_new.merge(fare_track_miles, how = 'left')
allvars_dataframe_new.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,track_miles,pairs,COMP_MILE,PEAK_FARE,OFF_PEAK_FARE,SD_FARE,TRAVEL_TIME,O_MSTN_ID,D_MSTN_ID,peak_fare_per_mile2
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,0.10,MSTN_0620MSTN_062,0.05,2.25,2.00,1.10,0,MSTN_062,MSTN_062,22.500000
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,...,10.35,MSTN_0010MSTN_062,8.03,3.80,3.15,1.90,31,MSTN_001,MSTN_062,0.367150
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,...,8.31,MSTN_0020MSTN_062,7.61,3.70,3.10,1.85,26,MSTN_002,MSTN_062,0.445247
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,...,11.39,MSTN_0650MSTN_062,10.26,4.45,3.65,2.20,35,MSTN_065,MSTN_062,0.390694
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,...,14.27,MSTN_0680MSTN_062,13.03,5.25,3.85,2.60,41,MSTN_068,MSTN_062,0.367905


In [15]:
allvars_dataframe_new.shape

(8281, 24)

## Off Peak Fare per Track Mile

* Subset only the station ID columns from fare_track_miles
* Create a unique OD pair column
* Merge into the main Off Peak dataframe

In [16]:
off_fare_track_miles= off_fare_track_miles[['O_MSTN_ID', 'D_MSTN_ID', 'off_peak_fare_per_mile']]
off_fare_track_miles['pairs'] = off_fare_track_miles.apply(lambda x: str(x['O_MSTN_ID'])+str(0)+str(x['D_MSTN_ID']), axis=1)

In [17]:
allvars_dataframe_new = allvars_dataframe_new.merge(off_fare_track_miles, how = 'left')
allvars_dataframe_new.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,pairs,COMP_MILE,PEAK_FARE,OFF_PEAK_FARE,SD_FARE,TRAVEL_TIME,O_MSTN_ID,D_MSTN_ID,peak_fare_per_mile2,off_peak_fare_per_mile
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,MSTN_0620MSTN_062,0.05,2.25,2.00,1.10,0,MSTN_062,MSTN_062,22.500000,20.000000
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,...,MSTN_0010MSTN_062,8.03,3.80,3.15,1.90,31,MSTN_001,MSTN_062,0.367150,0.304348
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,...,MSTN_0020MSTN_062,7.61,3.70,3.10,1.85,26,MSTN_002,MSTN_062,0.445247,0.373045
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,...,MSTN_0650MSTN_062,10.26,4.45,3.65,2.20,35,MSTN_065,MSTN_062,0.390694,0.320457
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,...,MSTN_0680MSTN_062,13.03,5.25,3.85,2.60,41,MSTN_068,MSTN_062,0.367905,0.269797


## AM Auto Travel Time & Auto Travel Time in Minutes/Mile

* Merge the interpolated auto travel times into the main AM dataframe

In [18]:
auto_tt_minpmile.head()

,pairs,new_auto_tt2,new_auto_tt_per_mile2
0,MSTN_0620MSTN_062,6.699843,66.998432
1,MSTN_0010MSTN_062,13.180000,1.641345
2,MSTN_0020MSTN_062,13.080000,1.718791
3,MSTN_0650MSTN_062,30.537346,2.681066
4,MSTN_0680MSTN_062,40.669998,3.116475


In [19]:
auto_tt_minpmile.rename(columns = {'new_auto_tt2':'am_new_auto_tt2' , 'new_auto_tt_per_mile2' :'am_new_auto_tt_per_mile2'}, inplace=True)
allvars_dataframe_new = allvars_dataframe_new.merge(auto_tt_minpmile, on='pairs', how = 'left')
allvars_dataframe_new.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,PEAK_FARE,OFF_PEAK_FARE,SD_FARE,TRAVEL_TIME,O_MSTN_ID,D_MSTN_ID,peak_fare_per_mile2,off_peak_fare_per_mile,am_new_auto_tt2,am_new_auto_tt_per_mile2
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,2.25,2.00,1.10,0,MSTN_062,MSTN_062,22.500000,20.000000,6.699843,66.998432
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,...,3.80,3.15,1.90,31,MSTN_001,MSTN_062,0.367150,0.304348,13.180000,1.641345
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,...,3.70,3.10,1.85,26,MSTN_002,MSTN_062,0.445247,0.373045,13.080000,1.718791
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,...,4.45,3.65,2.20,35,MSTN_065,MSTN_062,0.390694,0.320457,30.537346,2.681066
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,...,5.25,3.85,2.60,41,MSTN_068,MSTN_062,0.367905,0.269797,40.669998,3.116475


## PM Auto Travel Time & Auto Travel Time in Minutes/Mile

* Merge PM auto travel times into the main PM dataframe

In [20]:
pm_auto_tt.rename(columns = {'new_auto_tt2':'pm_new_auto_tt2' , 'new_auto_tt_per_mile2' :'pm_new_auto_tt_per_mile2'}, inplace=True)
allvars_dataframe_new = allvars_dataframe_new.merge(pm_auto_tt, on='pairs', how = 'left')
allvars_dataframe_new.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,SD_FARE,TRAVEL_TIME,O_MSTN_ID,D_MSTN_ID,peak_fare_per_mile2,off_peak_fare_per_mile,am_new_auto_tt2,am_new_auto_tt_per_mile2,pm_new_auto_tt2,pm_new_auto_tt_per_mile2
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,1.10,0,MSTN_062,MSTN_062,22.500000,20.000000,6.699843,66.998432,6.797840,67.978400
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,...,1.90,31,MSTN_001,MSTN_062,0.367150,0.304348,13.180000,1.641345,13.180000,1.641345
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,...,1.85,26,MSTN_002,MSTN_062,0.445247,0.373045,13.080000,1.718791,13.080000,1.718791
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,...,2.20,35,MSTN_065,MSTN_062,0.390694,0.320457,30.537346,2.681066,30.466196,2.674820
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,...,2.60,41,MSTN_068,MSTN_062,0.367905,0.269797,40.669998,3.116475,40.669998,3.116475


## Off Peak Auto Travel Time & Auto Travel Time in Minutes/Mile

* Merge into the main Off Peak dataframe

In [21]:
off_auto_tt.rename(columns = {'new_auto_tt2':'off_new_auto_tt2' , 'new_auto_tt_per_mile2' :'off_new_auto_tt_per_mile2'}, inplace = True)
allvars_dataframe_new = allvars_dataframe_new.merge(off_auto_tt, on='pairs', how = 'left')
allvars_dataframe_new.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,O_MSTN_ID,D_MSTN_ID,peak_fare_per_mile2,off_peak_fare_per_mile,am_new_auto_tt2,am_new_auto_tt_per_mile2,pm_new_auto_tt2,pm_new_auto_tt_per_mile2,off_new_auto_tt2,off_new_auto_tt_per_mile2
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,MSTN_062,MSTN_062,22.500000,20.000000,6.699843,66.998432,6.797840,67.978400,5.275970,52.759700
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,...,MSTN_001,MSTN_062,0.367150,0.304348,13.180000,1.641345,13.180000,1.641345,13.100000,1.265700
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,...,MSTN_002,MSTN_062,0.445247,0.373045,13.080000,1.718791,13.080000,1.718791,13.230000,1.592058
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,...,MSTN_065,MSTN_062,0.390694,0.320457,30.537346,2.681066,30.466196,2.674820,21.349543,1.874411
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,...,MSTN_068,MSTN_062,0.367905,0.269797,40.669998,3.116475,40.669998,3.116475,33.349998,2.337071


## Bus Travel Time & Bus Travel Time in Minutes/Mile

* Create a unique OD pair column
* Subset only the pairs, bus travel time and bus travel time per mile columns
* Merge into the main AM dataframe

In [22]:
bus_tt_minpmile['pairs'] = bus_tt_minpmile.apply(lambda x: str(x['O_MSTN_ID'])+str(0)+str(x['D_MSTN_ID']), axis=1)

In [23]:
bus_tt_minpmile2 = bus_tt_minpmile[['pairs', 'Travel Time', 'bus_tt_per_mile']]
bus_tt_minpmile2.rename(columns={'Travel Time':'bus_tt'}, inplace=True)

In [24]:
allvars_dataframe_new = allvars_dataframe_new.merge(bus_tt_minpmile2, on='pairs', how = 'left')
allvars_dataframe_new.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,peak_fare_per_mile2,off_peak_fare_per_mile,am_new_auto_tt2,am_new_auto_tt_per_mile2,pm_new_auto_tt2,pm_new_auto_tt_per_mile2,off_new_auto_tt2,off_new_auto_tt_per_mile2,bus_tt,bus_tt_per_mile
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,22.500000,20.000000,6.699843,66.998432,6.797840,67.978400,5.275970,52.759700,NaN,NaN
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,...,0.367150,0.304348,13.180000,1.641345,13.180000,1.641345,13.100000,1.265700,82.0,12.304868
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,...,0.445247,0.373045,13.080000,1.718791,13.080000,1.718791,13.230000,1.592058,84.0,11.415660
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,...,0.390694,0.320457,30.537346,2.681066,30.466196,2.674820,21.349543,1.874411,122.0,12.730069
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,...,0.367905,0.269797,40.669998,3.116475,40.669998,3.116475,33.349998,2.337071,180.0,14.449949


## AM Parking Users

* Give station names for the am_parking_users proper MSTN IDs
* Create a unique OD pairs column
* Subset only the pairs and the parking_user column
* Merge into the main AM dataframe

In [25]:
am_parking_users2 = mstns.merge(am_parking_users, left_on = 'WMATA_filename', right_on= 'START_PLACE_NAME')
am_parking_users2 = mstns.merge(am_parking_users2, left_on = 'WMATA_filename' , right_on='END_PLACE_NAME')
am_parking_users2.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,track_miles_names_y,auto_filename_y,Unnamed: 0,START_PLACE_NAME,END_PLACE_NAME,parking_user
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,Anacostia,ANACOSTIA,87,Anacostia,Addison Road,0.0
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,Ballston-MU,BALLSTON-MU,258,Ballston,Addison Road,0.0
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_003,Benning Road,BENNING ROAD,Benning Road,Benning Road,BENNING ROAD,348,Benning Road,Addison Road,0.0
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_052,Bethesda,BETHESDA,Bethesda,Bethesda,BETHESDA,437,Bethesda,Addison Road,0.0
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_078,Braddock Road,BRADDOCK ROAD,Braddock Road,Braddock Road,BRADDOCK ROAD,521,Braddock Road,Addison Road,0.0


In [26]:
am_parking_users2['pairs'] = am_parking_users2.apply(lambda x: str(x['ID_y'])+str(0)+str(x['ID_x']), axis=1)

In [27]:
am_parking_users2 = am_parking_users2[['pairs', 'parking_user']]
am_parking_users2.rename(columns = {'parking_user': 'am_parking_user'}, inplace = True)

In [28]:
allvars_dataframe_new = allvars_dataframe_new.merge(am_parking_users2, on='pairs', how='left')
allvars_dataframe_new.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,off_peak_fare_per_mile,am_new_auto_tt2,am_new_auto_tt_per_mile2,pm_new_auto_tt2,pm_new_auto_tt_per_mile2,off_new_auto_tt2,off_new_auto_tt_per_mile2,bus_tt,bus_tt_per_mile,am_parking_user
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,20.000000,6.699843,66.998432,6.797840,67.978400,5.275970,52.759700,NaN,NaN,NaN
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,...,0.304348,13.180000,1.641345,13.180000,1.641345,13.100000,1.265700,82.0,12.304868,0.0
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,...,0.373045,13.080000,1.718791,13.080000,1.718791,13.230000,1.592058,84.0,11.415660,NaN
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,...,0.320457,30.537346,2.681066,30.466196,2.674820,21.349543,1.874411,122.0,12.730069,NaN
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,...,0.269797,40.669998,3.116475,40.669998,3.116475,33.349998,2.337071,180.0,14.449949,0.0


## PM Parking Users

* Give station names in pm_parking_users proper MSTN IDs
* Create an OD pair column
* Subset only the pairs column and the parking_users
* Merge into the main PM dataframe

In [29]:
pm_parking_users2 = mstns.merge(pm_parking_users, left_on = 'WMATA_filename', right_on= 'START_PLACE_NAME')
pm_parking_users2 = mstns.merge(pm_parking_users2, left_on = 'WMATA_filename' , right_on='END_PLACE_NAME')
pm_parking_users2.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,track_miles_names_y,auto_filename_y,Unnamed: 0,START_PLACE_NAME,END_PLACE_NAME,parking_user
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,Anacostia,ANACOSTIA,76,Anacostia,Addison Road,0.0
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,163,Archives-Navy Memorial,Addison Road,14.0
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,Arlington Cemetery,ARLINGTON CEMETERY,253,Arlington Cemetery,Addison Road,0.0
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,Ballston-MU,BALLSTON-MU,337,Ballston,Addison Road,1.0
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_003,Benning Road,BENNING ROAD,Benning Road,Benning Road,BENNING ROAD,427,Benning Road,Addison Road,0.0


In [30]:
pm_parking_users2['pairs'] = pm_parking_users2.apply(lambda x: str(x['ID_y'])+str(0)+str(x['ID_x']), axis=1)

In [31]:
pm_parking_users2 = pm_parking_users2[['pairs', 'parking_user']]
pm_parking_users2.rename(columns = {'parking_user': 'pm_parking_user'}, inplace=True)

In [32]:
allvars_dataframe_new = allvars_dataframe_new.merge(pm_parking_users2, on='pairs', how='left')
allvars_dataframe_new.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,am_new_auto_tt2,am_new_auto_tt_per_mile2,pm_new_auto_tt2,pm_new_auto_tt_per_mile2,off_new_auto_tt2,off_new_auto_tt_per_mile2,bus_tt,bus_tt_per_mile,am_parking_user,pm_parking_user
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,6.699843,66.998432,6.797840,67.978400,5.275970,52.759700,NaN,NaN,NaN,NaN
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,...,13.180000,1.641345,13.180000,1.641345,13.100000,1.265700,82.0,12.304868,0.0,0.0
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,...,13.080000,1.718791,13.080000,1.718791,13.230000,1.592058,84.0,11.415660,NaN,14.0
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,...,30.537346,2.681066,30.466196,2.674820,21.349543,1.874411,122.0,12.730069,NaN,0.0
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,...,40.669998,3.116475,40.669998,3.116475,33.349998,2.337071,180.0,14.449949,0.0,1.0


## Off Peak Parking Users

* Give station names in pm_parking_users proper MSTN IDs
* Create an OD pair column
* Subset only the pairs column and the parking_users
* Merge into the main PM dataframe

In [33]:
off_parking_users2 = mstns.merge(off_parking_users, left_on = 'WMATA_filename', right_on= 'START_PLACE_NAME')
off_parking_users2 = mstns.merge(off_parking_users2, left_on = 'WMATA_filename' , right_on='END_PLACE_NAME')
off_parking_users2.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,track_miles_names_y,auto_filename_y,Unnamed: 0,START_PLACE_NAME,END_PLACE_NAME,parking_user
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,Anacostia,ANACOSTIA,87,Anacostia,Addison Road,0.0
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,177,Archives-Navy Memorial,Addison Road,4.0
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,Arlington Cemetery,ARLINGTON CEMETERY,266,Arlington Cemetery,Addison Road,0.0
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,Ballston-MU,BALLSTON-MU,343,Ballston,Addison Road,0.0
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_003,Benning Road,BENNING ROAD,Benning Road,Benning Road,BENNING ROAD,433,Benning Road,Addison Road,0.0


In [34]:
off_parking_users2['pairs'] = off_parking_users2.apply(lambda x: str(x['ID_y'])+str(0)+str(x['ID_x']), axis=1)

In [35]:
off_parking_users2 = off_parking_users2[['pairs', 'parking_user']]
off_parking_users2.rename(columns = {'parking_user': 'off_parking_user'}, inplace=True)

In [36]:
allvars_dataframe_new = allvars_dataframe_new.merge(off_parking_users2, on='pairs', how='left')
allvars_dataframe_new.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,am_new_auto_tt_per_mile2,pm_new_auto_tt2,pm_new_auto_tt_per_mile2,off_new_auto_tt2,off_new_auto_tt_per_mile2,bus_tt,bus_tt_per_mile,am_parking_user,pm_parking_user,off_parking_user
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,66.998432,6.797840,67.978400,5.275970,52.759700,NaN,NaN,NaN,NaN,NaN
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,...,1.641345,13.180000,1.641345,13.100000,1.265700,82.0,12.304868,0.0,0.0,0.0
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,...,1.718791,13.080000,1.718791,13.230000,1.592058,84.0,11.415660,NaN,14.0,4.0
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,...,2.681066,30.466196,2.674820,21.349543,1.874411,122.0,12.730069,NaN,0.0,0.0
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,...,3.116475,40.669998,3.116475,33.349998,2.337071,180.0,14.449949,0.0,1.0,0.0


In [37]:
allvars_dataframe_new.shape

(8281, 36)

## Bike travel time in minutes

In [38]:
bike_tt.head()

,name_O,name_D,median
0,ANACOSTIA,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,22.166667
1,ANACOSTIA,CAPITOL SOUTH,11.683333
2,ANACOSTIA,EASTERN MARKET,11.808333
3,ANACOSTIA,FARRAGUT NORTH,29.066667
4,ANACOSTIA,FARRAGUT WEST,29.066667


In [39]:
bike_tt2 = bike_tt.merge(mstns, left_on = 'name_O', right_on = 'walkshed_filename')
bike_tt2 = bike_tt2.merge(mstns, left_on = 'name_D', right_on = 'walkshed_filename')

In [40]:
bike_tt2

,name_O,name_D,median,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,track_miles_names_y,auto_filename_y
0,ANACOSTIA,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,22.166667,MSTN_001,Anacostia,ANACOSTIA,Anacostia,Anacostia,ANACOSTIA,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER
1,BALLSTON-MU,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,73.300000,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,Ballston-MU,BALLSTON-MU,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER
2,BROOKLAND-CUA,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,23.566667,MSTN_004,Brookland-CUA,BROOKLAND-CUA,Brookland,Brookland-CUA,BROOKLAND-CUA,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER
3,CAPITOL SOUTH,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,14.333333,MSTN_005,Capitol South,CAPITOL SOUTH,Capitol South,Capitol South,CAPITOL SOUTH,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER
4,CLARENDON,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,64.766667,MSTN_063,Clarendon,CLARENDON,Clarendon,Clarendon,CLARENDON,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,TYSONS CORNER,MCLEAN,13.300000,MSTN_088,Tysons,TYSONS CORNER,Tysons Corner,Tysons Corner,TYSONS,MSTN_087,McLean,MCLEAN,McLean,McLean,MCLEAN
1566,TYSONS CORNER,SPRING HILL,14.483333,MSTN_088,Tysons,TYSONS CORNER,Tysons Corner,Tysons Corner,TYSONS,MSTN_090,Spring Hill,SPRING HILL,Spring Hill,Spring Hill,SPRING HILL
1567,WEST HYATTSVILLE,PRINCE GEORGE'S PLAZA,17.716667,MSTN_055,West Hyattsville,WEST HYATTSVILLE,West Hyattsville,West Hyattsville,WEST HYATTSVILLE,MSTN_054,Hyattsville Crossing,PRINCE GEORGE'S PLAZA,Prince George's Plaza,Prince George's Plaza,HYATTSVILLE CROSSING
1568,WHITE FLINT,TWINBROOK,9.983333,MSTN_045,North Bethesda,WHITE FLINT,White Flint,White Flint,NORTH BETHESDA,MSTN_043,Twinbrook,TWINBROOK,Twinbrook,Twinbrook,TWINBROOK


In [41]:
bike_tt2['pairs'] = bike_tt2.apply(lambda x: str(x['ID_x'])+str(0)+str(x['ID_y']), axis=1)

In [42]:
bike_tt2['bike_traveltime'] = bike_tt2['median']
bike_tt2 = bike_tt2.drop('median', axis=1)
bike_tt2 = bike_tt2[['pairs', 'bike_traveltime']]

In [43]:
allvars_dataframe_new = allvars_dataframe_new.merge(bike_tt2, on = 'pairs', how='left')

In [44]:
allvars_dataframe_new.head(2)

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,pm_new_auto_tt2,pm_new_auto_tt_per_mile2,off_new_auto_tt2,off_new_auto_tt_per_mile2,bus_tt,bus_tt_per_mile,am_parking_user,pm_parking_user,off_parking_user,bike_traveltime
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,6.79784,67.978400,5.27597,52.7597,NaN,NaN,NaN,NaN,NaN,NaN
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,...,13.18000,1.641345,13.10000,1.2657,82.0,12.304868,0.0,0.0,0.0,NaN


In [45]:
allvars_dataframe_new.shape

(8281, 37)

## Google Bus Index & Auto Travel Times

In [46]:
bus_comp_index2 = bus_comp_index.merge(mstns, right_on = 'track_miles_names', left_on = 'station_name_o', how='left')
bus_comp_index2 = bus_comp_index2.merge(mstns, right_on = 'track_miles_names', left_on = 'station_name_d', how='left')

In [47]:
def o_func(row):    
    if row['station_name_o'] == 'McPherson Sq':
         return 'MSTN_023'
    elif row['station_name_o']  == 'Shaw-Howard U':
         return'MSTN_031'
    elif row['station_name_o']  == 'Virginia Sq-GMU':
         return 'MSTN_066'
    elif row['station_name_o']  == 'Wiehle-Reston East':
         return 'MSTN_091'
    elif row['station_name_o']  == 'Reston Town Center':
         return 'MSTN_092'
    elif row['station_name_o']  == 'Herndon':
         return 'MSTN_093'
    elif row['station_name_o']  == 'Dulles Airport':
         return 'MSTN_095'
    elif row['station_name_o']  == 'Loudoun Gateway':
         return 'MSTN_096'
    elif row['station_name_o']  == 'Innovation Center':
         return 'MSTN_094'
    elif row['station_name_o']  == 'Ashburn':
         return 'MSTN_097'

def d_func(row):    
    if row['station_name_d']  == 'McPherson Sq':   
        return 'MSTN_023'
    elif row['station_name_d']  == 'Shaw-Howard U':
         return'MSTN_031'
    elif row['station_name_d']  == 'Virginia Sq-GMU':
         return 'MSTN_066'
    elif row['station_name_d']  == 'Wiehle-Reston East':
         return 'MSTN_091'
    elif row['station_name_d']  == 'Reston Town Center':
         return 'MSTN_092'
    elif row['station_name_d']  == 'Herndon':
         return 'MSTN_093'
    elif row['station_name_d']  == 'Dulles Airport':
         return 'MSTN_095'
    elif row['station_name_d']  == 'Loudoun Gateway':
         return 'MSTN_096'
    elif row['station_name_d']  == 'Innovation Center':
         return 'MSTN_094'
    elif row['station_name_d']  == 'Ashburn':
         return 'MSTN_097'

In [48]:
bus_comp_index2['missing_o'] = bus_comp_index2.apply(o_func, axis=1)
bus_comp_index2['missing_d'] = bus_comp_index2.apply(d_func, axis=1)
bus_comp_index2['ID_x'] = bus_comp_index2['ID_x'].fillna(bus_comp_index2['missing_o'])
bus_comp_index2['ID_y'] = bus_comp_index2['ID_y'].fillna(bus_comp_index2['missing_d'])
google_auto_times2 = google_auto_times.merge(mstns, right_on = 'track_miles_names', left_on = 'station_name_o', how='left')
google_auto_times2 = google_auto_times2.merge(mstns, right_on = 'track_miles_names', left_on = 'station_name_d', how='left')
google_auto_times2['missing_o'] = google_auto_times2.apply(o_func, axis=1)
google_auto_times2['missing_d'] = google_auto_times2 .apply(d_func, axis=1)
google_auto_times2['ID_x'] = google_auto_times2['ID_x'].fillna(google_auto_times2['missing_o'])
google_auto_times2['ID_y'] = google_auto_times2['ID_y'].fillna(google_auto_times2['missing_d'])
google_auto_times2['pairs'] = google_auto_times2.apply(lambda x: str(x['ID_x'])+str(0)+str(x['ID_y']), axis=1)
bus_comp_index2['pairs'] = bus_comp_index2.apply(lambda x: str(x['ID_x'])+str(0)+str(x['ID_y']), axis=1)

In [49]:
google_auto_times2 = google_auto_times2[['pairs', 'google_driving_minutes', 'google_driving_miles']]
bus_comp_index2 = bus_comp_index2[['pairs', 'bus_transit_minutes',
       'metro_transit_minutes', 'bus_competativeness_index']]

In [50]:
allvars_dataframe_new = allvars_dataframe_new.merge(google_auto_times2, on = 'pairs', how = 'left')
allvars_dataframe_new = allvars_dataframe_new.merge(bus_comp_index2, on = 'pairs', how='left')
allvars_dataframe_new.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,bus_tt_per_mile,am_parking_user,pm_parking_user,off_parking_user,bike_traveltime,google_driving_minutes,google_driving_miles,bus_transit_minutes,metro_transit_minutes,bus_competativeness_index
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.000000
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,...,12.304868,0.0,0.0,0.0,NaN,18.05,7.59,35.82,31,0.865438
2,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,...,11.415660,NaN,14.0,4.0,NaN,22.35,9.97,64.18,26,0.405111
3,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,...,12.730069,NaN,0.0,0.0,NaN,25.43,13.10,103.58,35,0.337903
4,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,...,14.449949,0.0,1.0,0.0,NaN,32.33,15.02,121.37,41,0.337810


## Metro Swipecard Travel Time 

In [51]:
metro_swipe_tt.head(2)

,pairs,trip_time_mins
0,MSTN_0010MSTN_002,18.423072
1,MSTN_0010MSTN_003,40.364957


In [52]:
allvars_dataframe_new = allvars_dataframe_new.merge(metro_swipe_tt, on = 'pairs', how = 'left')

In [53]:
allvars_dataframe_new.shape

(8281, 43)

# .5 Station Level Variables

## AM Trains per Hour

* Filter only for weekday services and in the AM peak
* Groupby and get the sum of trains per hour
* Merge into the main AM dataframe

In [54]:
amtrains_per_hour2 = trains_per_hour[(trains_per_hour['ARRIVAL_PERIOD']=='AM Peak') & (trains_per_hour['SERVICETYPE']=='Weekday')]
amtrains_per_hour2.head()

,STATION_ID,STOP_ID,DIR_ID,SERVICETYPE,ARRIVAL_PERIOD,AVG_TRAINS
8,MSTN_001,F06,1,Weekday,AM Peak,17
10,MSTN_001,F06,2,Weekday,AM Peak,16
20,MSTN_002,F02,1,Weekday,AM Peak,31
22,MSTN_002,F02,2,Weekday,AM Peak,31
32,MSTN_003,G01,1,Weekday,AM Peak,24


In [55]:
amtrains_per_hour2 = amtrains_per_hour2.groupby(['STATION_ID']).sum().reset_index()
amtrains_per_hour2 = amtrains_per_hour2[['STATION_ID','AVG_TRAINS']]
amtrains_per_hour2.rename(columns = {'AVG_TRAINS': 'AM_AVG_TRAINS'}, inplace=True)
amtrains_per_hour2.head()

,STATION_ID,AM_AVG_TRAINS
0,MSTN_001,33
1,MSTN_002,62
2,MSTN_003,51
3,MSTN_004,56
4,MSTN_005,78


In [56]:
stationlevel_5mile = mstns.merge(amtrains_per_hour2, left_on = 'ID', right_on = 'STATION_ID', how = 'left').drop(['STATION_ID','PRIMARY_NAME_FY23', 'walkshed_filename', 'WMATA_filename', 'track_miles_names', 'auto_filename'	 ], axis =1)
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS
0,MSTN_062,53.0
1,MSTN_001,33.0
2,MSTN_002,62.0
3,MSTN_065,27.0
4,MSTN_097,NaN


## PM Trains per Hour

* Filter only for weekday services and in the PM peak
* Groupby and get the sum of trains per hour
* Merge into the PM dataframe first based on the origins and again based on the destinations
* Drop redundant columns

In [57]:
pmtrains_per_hour2 = trains_per_hour[(trains_per_hour['ARRIVAL_PERIOD']=='PM Peak') & (trains_per_hour['SERVICETYPE']=='Weekday')]
pmtrains_per_hour2.head()

,STATION_ID,STOP_ID,DIR_ID,SERVICETYPE,ARRIVAL_PERIOD,AVG_TRAINS
9,MSTN_001,F06,1,Weekday,PM Peak,17
11,MSTN_001,F06,2,Weekday,PM Peak,17
21,MSTN_002,F02,1,Weekday,PM Peak,31
23,MSTN_002,F02,2,Weekday,PM Peak,31
33,MSTN_003,G01,1,Weekday,PM Peak,24


In [58]:
pmtrains_per_hour2 = pmtrains_per_hour2.groupby(['STATION_ID']).sum().reset_index()
pmtrains_per_hour2 = pmtrains_per_hour2[['STATION_ID','AVG_TRAINS']]
pmtrains_per_hour2.rename(columns = {'AVG_TRAINS': 'PM_AVG_TRAINS'}, inplace = True)
pmtrains_per_hour2.head()

,STATION_ID,PM_AVG_TRAINS
0,MSTN_001,34
1,MSTN_002,62
2,MSTN_003,48
3,MSTN_004,53
4,MSTN_005,74


In [59]:
stationlevel_5mile = stationlevel_5mile.merge(pmtrains_per_hour2, left_on = 'ID', right_on='STATION_ID', how = 'left')

In [60]:
stationlevel_5mile.head() 

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS
0,MSTN_062,53.0,MSTN_062,48.0
1,MSTN_001,33.0,MSTN_001,34.0
2,MSTN_002,62.0,MSTN_002,62.0
3,MSTN_065,27.0,MSTN_065,24.0
4,MSTN_097,NaN,NaN,NaN


## All Periods, Trains per Hour

In [61]:
all_trains_per_hour.head()

,STATION_ID,AVG_TRAINS
0,MSTN_001,3.400000
1,MSTN_002,6.340909
2,MSTN_003,5.318182
3,MSTN_004,5.311111
4,MSTN_005,8.186047


In [62]:
all_trains_per_hour.rename(columns = {'AVG_TRAINS': 'ALLPERIODS_AVG_TRAINS', 'STATION_ID':'ID'}, inplace = True)
stationlevel_5mile = stationlevel_5mile.merge(all_trains_per_hour, left_on = 'ID', right_on='ID', how = 'left')

In [63]:
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS
0,MSTN_062,53.0,MSTN_062,48.0,5.340909
1,MSTN_001,33.0,MSTN_001,34.0,3.400000
2,MSTN_002,62.0,MSTN_002,62.0,6.340909
3,MSTN_065,27.0,MSTN_065,24.0,2.904762
4,MSTN_097,NaN,NaN,NaN,NaN


## Households per Half Mile

* Subset only the ID, proportion of houses and the total number of households. Rename ID to MSTN
* Merge once based on the destinations and again based on the origins
* Drop duplicate columns

In [64]:
households_half_mile.head()

,Unnamed: 0,Name_1,proportionhouses,Total Households,ID,PRIMARY_NAME_FY23
0,0,Addison Road,0.528921,1663,MSTN_062,Addison Road
1,1,Anacostia,2.075704,3639,MSTN_001,Anacostia
2,2,Archives,2.214692,3007,MSTN_002,Archives
3,3,Arlington Cemetery,0.000003,1,MSTN_065,Arlington Cemetery
4,4,Ashburn,0.555594,936,MSTN_097,Ashburn


In [65]:
households_half_mile = households_half_mile[['ID', 'proportionhouses', 'Total Households', ]]
households_half_mile.rename(columns={'ID':'MSTN'}, inplace=True)

In [66]:
households_half_mile.head()

,MSTN,proportionhouses,Total Households
0,MSTN_062,0.528921,1663
1,MSTN_001,2.075704,3639
2,MSTN_002,2.214692,3007
3,MSTN_065,0.000003,1
4,MSTN_097,0.555594,936


In [67]:
stationlevel_5mile = stationlevel_5mile.merge(households_half_mile, right_on = 'MSTN', left_on='ID', how = 'left').drop(['MSTN'], axis = 1)
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0


## Parking Capacity

* Drop the station column from parking_capacity and rename the MSTN_ID column to avoid errors while merging
* Merge into the AM dataframe first based on the origins and again based on the destinations
* Fill NAs in parking capacity with 0

In [68]:
parking_capacity.drop('STATION', axis=1, inplace=True)

In [69]:
stationlevel_5mile = stationlevel_5mile.merge(parking_capacity, right_on = 'MSTN_ID', left_on='ID', how = 'left').drop(['MSTN_ID'], axis = 1)
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0


## Number of Bus Lines

* Rename the bus line count and bus stop columns to distinguish them
* Merge into the main AM dataframe, first based on destinations and again based on origins
* Drop columns that are redundant

In [70]:
num_bus_lines.head()

,Name_1,MSTN,line_count_p5,stop_count_p5
0,ADDISON ROAD-SEAT PLEASANT,MSTN_062,36.0,18.0
1,ANACOSTIA,MSTN_001,64.0,36.0
2,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,MSTN_002,86.0,50.0
3,ARLINGTON CEMETERY,MSTN_065,9.0,0.0
4,BALLSTON-MU,MSTN_068,35.0,30.0


In [71]:
num_bus_lines.rename(columns={'line_count_p5':'bus_line_count','stop_count_p5':'bus_stop_count'}, inplace=True)
num_bus_lines = num_bus_lines[['MSTN','bus_line_count','bus_stop_count']]

In [72]:
stationlevel_5mile = stationlevel_5mile.merge(num_bus_lines, right_on = 'MSTN', left_on='ID', how = 'left').drop(['MSTN'], axis = 1)
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0


## Jobs per Half Mile

* Merge Jobs into the AM dataframe first based on the origins and again based on the destinations

In [73]:
jobs_half_mile2 = mstns.merge(jobs_half_mile, left_on = 'walkshed_filename', right_on='Name_1').drop(['PRIMARY_NAME_FY23', 'walkshed_filename', 'WMATA_filename',
       'track_miles_names', 'auto_filename', 'Name_1'], axis = 1)

In [74]:
stationlevel_5mile = stationlevel_5mile.merge(jobs_half_mile2, right_on = 'ID', left_on='ID', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p5
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,205.510639
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,2332.571897
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,0.000000
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,122.300629
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,1306.797305


## Distance to the Core

* Give the station names in dist_to_core proper formatting to merge with mstns
* Give station names proper MSTN IDs
* Subset only the ID and distance_miles columns. Rename both for easy merging and clarity
* Merge into the AM dataframe first based on destinations, then based on origins

In [75]:
dist_to_core['STATIONNAM'] = dist_to_core['STATIONNAM'].str.replace(' : 0 - 2640','')
dist_to_core['STATIONNAM'] = dist_to_core['STATIONNAM'].str.replace(' : 0 - 22.4525758392805','')
dist_to_core['STATIONNAM'] = dist_to_core['STATIONNAM'].str.title()
dist_to_core['STATIONNAM'] = dist_to_core['STATIONNAM'].replace({"Addison Road-Seat Pleasant": 'Addison Road', "Archives-Navy Mem'L-Penn Quarter":"Archives", "Ballston-Mu": "Ballston-MU", "Brookland-Cua":"Brookland-CUA", "Dunn Loring-Merrifield": "Dunn Loring", "Largo Town Center":"Downtown Largo", "Eisenhower Avenue":"Eisenhower Ave", 'Federal Center Sw':'Federal Center SW', "Foggy Bottom-Gwu":"Foggy Bottom-GWU","Gallery Pl-Chinatown":"Gallary Place", "Mclean":"McLean", "Mcpherson Square":"McPherson Sq", "Mt Vernon Sq/7Th St-Convention Center":"Mt Vernon Sq", "Noma-Gallaudet U-New York Ave":"NoMa-Gallaudet U", "Rhode Island Ave-Brentwood":"Rhode Island Ave",  "Prince George'S Plaza":"Hyattsville Crossing", "Tenleytown-Au": "Tenleytown-AU", "Tysons Corner":"Tysons", "U St/African-Amer Civil War Memorial/Cardozo": "U Street", "Southern Avenue": "Southern Ave", "Van Ness-Udc":"Van Ness-UDC", "Virginia Square-Gmu":"Virginia Sq-GMU", "Vienna/Fairfax-Gmu":"Vienna", "Washington Dulles International Airport":"Dulles Airport", "West Falls Church-Vt/Uva":"West Falls Church", "Woodley Park-Zoo/Adams Morgan":"Woodley Park", "White Flint":"North Bethesda" })
dist_to_core['STATIONNAM'] = dist_to_core['STATIONNAM'].replace({"College Park-U Of Md": 'College Park-U of Md', "Gallary Place": "Gallery Place", "Shaw-Howard Univ":"Shaw-Howard U", "Innovation":"Innovation Center", "Reston":"Reston Town Center"})

In [76]:
dist_to_core2 = mstns.merge(dist_to_core, left_on = 'PRIMARY_NAME_FY23', right_on= 'STATIONNAM')

In [77]:
dist_to_core2 = dist_to_core2[['ID','distance_miles']]
dist_to_core2.rename(columns={'ID':'id1', 'distance_miles':'distance_to_core'}, inplace=True)

In [78]:
stationlevel_5mile = stationlevel_5mile.merge(dist_to_core2, right_on = 'id1', left_on='ID', how = 'left').drop(['id1'], axis = 1)
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p5,distance_to_core
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,205.510639,7.181143
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,2332.571897,3.007895
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,0.000000,0.435964
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,122.300629,2.127338
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,1306.797305,25.994746


## Night and Weekend Jobs

* Merge into the Off Peak dataframe first based on the origins and again based on the destinations

In [79]:
nite_wkend_jobs

,Name_1,Proportion_night_weekend_jobs_p5
0,ADDISON ROAD-SEAT PLEASANT,10.449694
1,ANACOSTIA,134.784916
2,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,0.000000
3,ARLINGTON CEMETERY,63.155243
4,Ashburn,471.516448
...,...,...
92,WHEATON,1388.708404
93,WHITE FLINT,2257.603743
94,WIEHLE-RESTON EAST,0.000000
95,WOODLEY PARK-ZOO/ADAMS MORGAN,3147.825035


In [80]:
nite_wkend_jobs2 = mstns.merge(nite_wkend_jobs, left_on = 'walkshed_filename', right_on= 'Name_1')
nite_wkend_jobs2 = nite_wkend_jobs2[['ID', 'Name_1', 'Proportion_night_weekend_jobs_p5']]

In [81]:
stationlevel_5mile = stationlevel_5mile.merge(nite_wkend_jobs2, right_on = 'ID', left_on='ID', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p5,distance_to_core,Name_1,Proportion_night_weekend_jobs_p5
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,205.510639,7.181143,ADDISON ROAD-SEAT PLEASANT,10.449694
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,2332.571897,3.007895,ANACOSTIA,134.784916
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,0.000000,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,0.000000
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,122.300629,2.127338,ARLINGTON CEMETERY,63.155243
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,1306.797305,25.994746,Ashburn,471.516448


## 9-5 Jobs

In [82]:
nine_five_jobs

,Name_1,Total_Nine_to_Five_workers_p5
0,ADDISON ROAD-SEAT PLEASANT,0.000000
1,ANACOSTIA,344.097962
2,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,0.000000
3,ARLINGTON CEMETERY,51.125673
4,Ashburn,629.498764
...,...,...
92,WHEATON,261.527006
93,WHITE FLINT,4019.849772
94,WIEHLE-RESTON EAST,15.195713
95,WOODLEY PARK-ZOO/ADAMS MORGAN,514.738685


In [83]:
stationlevel_5mile = stationlevel_5mile.merge(nine_five_jobs, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p5,distance_to_core,Name_1,Proportion_night_weekend_jobs_p5,Total_Nine_to_Five_workers_p5
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,205.510639,7.181143,ADDISON ROAD-SEAT PLEASANT,10.449694,0.000000
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,2332.571897,3.007895,ANACOSTIA,134.784916,344.097962
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,0.000000,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,0.000000,0.000000
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,122.300629,2.127338,ARLINGTON CEMETERY,63.155243,51.125673
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,1306.797305,25.994746,Ashburn,471.516448,629.498764


## School Jobs

In [84]:
school_jobs

,Name_1,Proportion_education_jobs_p5
0,ADDISON ROAD-SEAT PLEASANT,0.000000
1,ANACOSTIA,415.454682
2,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,0.000000
3,ARLINGTON CEMETERY,0.000000
4,Ashburn,19.443977
...,...,...
92,WHEATON,0.000000
93,WHITE FLINT,87.673932
94,WIEHLE-RESTON EAST,23.878977
95,WOODLEY PARK-ZOO/ADAMS MORGAN,217.774059


In [85]:
stationlevel_5mile = stationlevel_5mile.merge(school_jobs, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p5,distance_to_core,Name_1,Proportion_night_weekend_jobs_p5,Total_Nine_to_Five_workers_p5,Proportion_education_jobs_p5
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,205.510639,7.181143,ADDISON ROAD-SEAT PLEASANT,10.449694,0.000000,0.000000
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,2332.571897,3.007895,ANACOSTIA,134.784916,344.097962,415.454682
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,0.000000,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,0.000000,0.000000,0.000000
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,122.300629,2.127338,ARLINGTON CEMETERY,63.155243,51.125673,0.000000
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,1306.797305,25.994746,Ashburn,471.516448,629.498764,19.443977


## Median Household Income

* Rename the station name column to make it easier for merging 
* Merge into the Off Peak dataframe first based on the origins and again based on the destinations
* Fill any NAs with 0

In [86]:
stationlevel_5mile = stationlevel_5mile.merge(median_income, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p5,distance_to_core,Name_1,Proportion_night_weekend_jobs_p5,Total_Nine_to_Five_workers_p5,Proportion_education_jobs_p5,Median_household_income_p5
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,205.510639,7.181143,ADDISON ROAD-SEAT PLEASANT,10.449694,0.000000,0.000000,77702.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,2332.571897,3.007895,ANACOSTIA,134.784916,344.097962,415.454682,36323.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,0.000000,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,0.000000,0.000000,0.000000,135011.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,122.300629,2.127338,ARLINGTON CEMETERY,63.155243,51.125673,0.000000,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,1306.797305,25.994746,Ashburn,471.516448,629.498764,19.443977,135179.0


## Terminal Station Dummy

* Subset only the dummy column and the station IDs
* Merge into the AM dataframe once based on destinations and again based on origins

In [87]:
terminal_station2 = terminal_station[['MSTN','terminal_dummy_2023']]

In [88]:
stationlevel_5mile = stationlevel_5mile.merge(terminal_station2, right_on = 'MSTN', left_on='ID', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p5,distance_to_core,Name_1,Proportion_night_weekend_jobs_p5,Total_Nine_to_Five_workers_p5,Proportion_education_jobs_p5,Median_household_income_p5,MSTN,terminal_dummy_2023
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,205.510639,7.181143,ADDISON ROAD-SEAT PLEASANT,10.449694,0.000000,0.000000,77702.0,MSTN_062,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,2332.571897,3.007895,ANACOSTIA,134.784916,344.097962,415.454682,36323.0,MSTN_001,NaN
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,0.000000,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,0.000000,0.000000,0.000000,135011.0,MSTN_002,NaN
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,122.300629,2.127338,ARLINGTON CEMETERY,63.155243,51.125673,0.000000,NaN,MSTN_065,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,1306.797305,25.994746,Ashburn,471.516448,629.498764,19.443977,135179.0,MSTN_097,1.0


## Bikeshare Capacity

In [89]:
bikeshare_cap.head()

,Name_1,CAPACITY_p5
0,ANACOSTIA,30
1,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,324
2,BALLSTON-MU,133
3,BENNING ROAD,19
4,BETHESDA,90


In [90]:
bikeshare_cap['bike_cap'] = bikeshare_cap['CAPACITY_p5']
bikeshare_cap = bikeshare_cap.drop('CAPACITY_p5', axis=1)

In [91]:
stationlevel_5mile = stationlevel_5mile.merge(bikeshare_cap, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p5,distance_to_core,Name_1,Proportion_night_weekend_jobs_p5,Total_Nine_to_Five_workers_p5,Proportion_education_jobs_p5,Median_household_income_p5,MSTN,terminal_dummy_2023,bike_cap
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,205.510639,7.181143,ADDISON ROAD-SEAT PLEASANT,10.449694,0.000000,0.000000,77702.0,MSTN_062,NaN,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,2332.571897,3.007895,ANACOSTIA,134.784916,344.097962,415.454682,36323.0,MSTN_001,NaN,30.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,0.000000,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,0.000000,0.000000,0.000000,135011.0,MSTN_002,NaN,324.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,122.300629,2.127338,ARLINGTON CEMETERY,63.155243,51.125673,0.000000,NaN,MSTN_065,NaN,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,1306.797305,25.994746,Ashburn,471.516448,629.498764,19.443977,135179.0,MSTN_097,1.0,NaN


## CTPP Jobs

In [92]:
# pd.set_option('display.max_rows', None)
# pd.options.display.max_rows = None
# pd.reset_option('all')
ctpp_jobs = ctpp_jobs[['Name_1', 'proportionjobs_p5' ]].rename(columns={'proportionjobs_p5': 'ctpp_jobs'}).copy()

In [93]:
ctpp_jobs.head(2)

,Name_1,ctpp_jobs
0,ADDISON ROAD-SEAT PLEASANT,116.608128
1,ANACOSTIA,1763.609330


In [94]:
stationlevel_5mile = stationlevel_5mile.merge(ctpp_jobs, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,distance_to_core,Name_1,Proportion_night_weekend_jobs_p5,Total_Nine_to_Five_workers_p5,Proportion_education_jobs_p5,Median_household_income_p5,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,7.181143,ADDISON ROAD-SEAT PLEASANT,10.449694,0.000000,0.000000,77702.0,MSTN_062,NaN,NaN,116.608128
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,3.007895,ANACOSTIA,134.784916,344.097962,415.454682,36323.0,MSTN_001,NaN,30.0,1763.609330
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,0.000000,0.000000,0.000000,135011.0,MSTN_002,NaN,324.0,56593.758076
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,2.127338,ARLINGTON CEMETERY,63.155243,51.125673,0.000000,NaN,MSTN_065,NaN,NaN,35.889127
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,25.994746,Ashburn,471.516448,629.498764,19.443977,135179.0,MSTN_097,1.0,NaN,278.607874


## Household Income below 50k

In [95]:
hh_incomebelowpoverty.head(2)

,Name_1,HH_below_50k_p5
0,ADDISON ROAD-SEAT PLEASANT,922.4
1,ANACOSTIA,2932.3


In [96]:
stationlevel_5mile = stationlevel_5mile.merge(hh_incomebelowpoverty, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,Name_1,Proportion_night_weekend_jobs_p5,Total_Nine_to_Five_workers_p5,Proportion_education_jobs_p5,Median_household_income_p5,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p5
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,ADDISON ROAD-SEAT PLEASANT,10.449694,0.000000,0.000000,77702.0,MSTN_062,NaN,NaN,116.608128,922.4
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,ANACOSTIA,134.784916,344.097962,415.454682,36323.0,MSTN_001,NaN,30.0,1763.609330,2932.3
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,0.000000,0.000000,0.000000,135011.0,MSTN_002,NaN,324.0,56593.758076,548.4
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,ARLINGTON CEMETERY,63.155243,51.125673,0.000000,NaN,MSTN_065,NaN,NaN,35.889127,0.0
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,Ashburn,471.516448,629.498764,19.443977,135179.0,MSTN_097,1.0,NaN,278.607874,193.1


## Number of hotels within the walkshed

In [97]:
hotels.head(2)

,Name_1,hotelcount_p5
0,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,4
1,BETHESDA,1


In [98]:
stationlevel_5mile = stationlevel_5mile.merge(hotels, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,Proportion_night_weekend_jobs_p5,Total_Nine_to_Five_workers_p5,Proportion_education_jobs_p5,Median_household_income_p5,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p5,hotelcount_p5
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,10.449694,0.000000,0.000000,77702.0,MSTN_062,NaN,NaN,116.608128,922.4,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,134.784916,344.097962,415.454682,36323.0,MSTN_001,NaN,30.0,1763.609330,2932.3,NaN
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,0.000000,0.000000,0.000000,135011.0,MSTN_002,NaN,324.0,56593.758076,548.4,4.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,63.155243,51.125673,0.000000,NaN,MSTN_065,NaN,NaN,35.889127,0.0,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,471.516448,629.498764,19.443977,135179.0,MSTN_097,1.0,NaN,278.607874,193.1,NaN


## Number of entertainment locations within the walkshed

In [99]:
entertainment.head()

,Name_1,entertainmentcount_p5
0,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,8
1,BALLSTON-MU,1
2,BETHESDA,3
3,BROOKLAND-CUA,4
4,CAPITOL SOUTH,1


In [100]:
stationlevel_5mile = stationlevel_5mile.merge(entertainment, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,Total_Nine_to_Five_workers_p5,Proportion_education_jobs_p5,Median_household_income_p5,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p5,hotelcount_p5,entertainmentcount_p5
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,0.000000,0.000000,77702.0,MSTN_062,NaN,NaN,116.608128,922.4,NaN,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,344.097962,415.454682,36323.0,MSTN_001,NaN,30.0,1763.609330,2932.3,NaN,NaN
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,0.000000,0.000000,135011.0,MSTN_002,NaN,324.0,56593.758076,548.4,4.0,8.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,51.125673,0.000000,NaN,MSTN_065,NaN,NaN,35.889127,0.0,NaN,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,629.498764,19.443977,135179.0,MSTN_097,1.0,NaN,278.607874,193.1,NaN,NaN


## Number of restaurants located within each walksheds

In [101]:
restaurant.head(2)

,Name_1,restaurantcount_p5
0,ANACOSTIA,4
1,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,76


In [102]:
stationlevel_5mile = stationlevel_5mile.merge(restaurant, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,Proportion_education_jobs_p5,Median_household_income_p5,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p5,hotelcount_p5,entertainmentcount_p5,restaurantcount_p5
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,0.000000,77702.0,MSTN_062,NaN,NaN,116.608128,922.4,NaN,NaN,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,415.454682,36323.0,MSTN_001,NaN,30.0,1763.609330,2932.3,NaN,NaN,4.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,0.000000,135011.0,MSTN_002,NaN,324.0,56593.758076,548.4,4.0,8.0,76.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,0.000000,NaN,MSTN_065,NaN,NaN,35.889127,0.0,NaN,NaN,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,19.443977,135179.0,MSTN_097,1.0,NaN,278.607874,193.1,NaN,NaN,3.0


## Station is an Inter-city hub (by ridership)

In [103]:
inter_city.head(2)

,Name_1,ridership_p5
0,ADDISON ROAD,19259.0
1,ANACOSTIA,40970.0


In [104]:
inter_city['intercityhub'] = inter_city['ridership_p5']
inter_city.drop('ridership_p5', axis=1, inplace=True)

In [105]:
stationlevel_5mile = stationlevel_5mile.merge(inter_city, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,Median_household_income_p5,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p5,hotelcount_p5,entertainmentcount_p5,restaurantcount_p5,intercityhub
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,77702.0,MSTN_062,NaN,NaN,116.608128,922.4,NaN,NaN,NaN,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,36323.0,MSTN_001,NaN,30.0,1763.609330,2932.3,NaN,NaN,4.0,40970.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,135011.0,MSTN_002,NaN,324.0,56593.758076,548.4,4.0,8.0,76.0,NaN
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,NaN,MSTN_065,NaN,NaN,35.889127,0.0,NaN,NaN,NaN,27714.0
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,135179.0,MSTN_097,1.0,NaN,278.607874,193.1,NaN,NaN,3.0,NaN


## Intersection density

In [106]:
intersection_density.head(2)

,Name_1,proportional_street_density_p5
0,ADDISON ROAD-SEAT PLEASANT,87.782487
1,ANACOSTIA,71.775767


In [107]:
intersection_density['prop_str_dens'] = intersection_density['proportional_street_density_p5']
intersection_density.drop('proportional_street_density_p5', axis=1, inplace=True)

In [108]:
stationlevel_5mile = stationlevel_5mile.merge(intersection_density, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p5,hotelcount_p5,entertainmentcount_p5,restaurantcount_p5,intercityhub,prop_str_dens
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,MSTN_062,NaN,NaN,116.608128,922.4,NaN,NaN,NaN,NaN,87.782487
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,MSTN_001,NaN,30.0,1763.609330,2932.3,NaN,NaN,4.0,40970.0,71.775767
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,MSTN_002,NaN,324.0,56593.758076,548.4,4.0,8.0,76.0,NaN,170.832426
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,MSTN_065,NaN,NaN,35.889127,0.0,NaN,NaN,NaN,27714.0,54.509764
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,MSTN_097,1.0,NaN,278.607874,193.1,NaN,NaN,3.0,NaN,5.291906


## Job Accessibility by Transit

In [109]:
job_access_by_transit.head(2)

,Name_1,Jobs_auto,Jobs_transit_p5
0,ADDISON ROAD-SEAT PLEASANT,22677332.0,15074627.0
1,ANACOSTIA,41573746.0,47484669.0


In [110]:
job_access_by_transit = job_access_by_transit[['Name_1', 'Jobs_transit_p5']]

In [111]:
stationlevel_5mile = stationlevel_5mile.merge(job_access_by_transit, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p5,hotelcount_p5,entertainmentcount_p5,restaurantcount_p5,intercityhub,prop_str_dens,Jobs_transit_p5
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,NaN,NaN,116.608128,922.4,NaN,NaN,NaN,NaN,87.782487,15074627.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,NaN,30.0,1763.609330,2932.3,NaN,NaN,4.0,40970.0,71.775767,47484669.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,NaN,324.0,56593.758076,548.4,4.0,8.0,76.0,NaN,170.832426,13524248.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,NaN,NaN,35.889127,0.0,NaN,NaN,NaN,27714.0,54.509764,0.0
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,1.0,NaN,278.607874,193.1,NaN,NaN,3.0,NaN,5.291906,0.0


## Number of convenience stores within each walksheds

In [112]:
convenience_store.head(2)

,Name_1,convenience_storecount_p5
0,ANACOSTIA,3
1,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,1


In [113]:
stationlevel_5mile = stationlevel_5mile.merge(convenience_store, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,bike_cap,ctpp_jobs,HH_below_50k_p5,hotelcount_p5,entertainmentcount_p5,restaurantcount_p5,intercityhub,prop_str_dens,Jobs_transit_p5,convenience_storecount_p5
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,NaN,116.608128,922.4,NaN,NaN,NaN,NaN,87.782487,15074627.0,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,30.0,1763.609330,2932.3,NaN,NaN,4.0,40970.0,71.775767,47484669.0,3.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,324.0,56593.758076,548.4,4.0,8.0,76.0,NaN,170.832426,13524248.0,1.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,NaN,35.889127,0.0,NaN,NaN,NaN,27714.0,54.509764,0.0,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,NaN,278.607874,193.1,NaN,NaN,3.0,NaN,5.291906,0.0,NaN


## High School Enrollment

In [114]:
highschool_enrollment.head(2)

,Name_1,TOTAL_p5
0,ANACOSTIA,1593.0
1,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,728.0


In [115]:
highschool_enrollment['highschoolenroll'] = highschool_enrollment['TOTAL_p5']
highschool_enrollment.drop('TOTAL_p5', axis=1, inplace=True)

In [116]:
stationlevel_5mile = stationlevel_5mile.merge(highschool_enrollment, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,ctpp_jobs,HH_below_50k_p5,hotelcount_p5,entertainmentcount_p5,restaurantcount_p5,intercityhub,prop_str_dens,Jobs_transit_p5,convenience_storecount_p5,highschoolenroll
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,116.608128,922.4,NaN,NaN,NaN,NaN,87.782487,15074627.0,NaN,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,1763.609330,2932.3,NaN,NaN,4.0,40970.0,71.775767,47484669.0,3.0,1593.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,56593.758076,548.4,4.0,8.0,76.0,NaN,170.832426,13524248.0,1.0,728.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,35.889127,0.0,NaN,NaN,NaN,27714.0,54.509764,0.0,NaN,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,278.607874,193.1,NaN,NaN,3.0,NaN,5.291906,0.0,NaN,NaN


## College Enrollment within the walkshed

In [117]:
college_enrollment.head()

,Name_1,TOT_ENROLL_p5
0,BROOKLAND-CUA,5790.0
1,CLARENDON,8.0
2,COURT HOUSE,4797.0
3,CRYSTAL CITY,580.0
4,DUPONT CIRCLE,340.0


In [118]:
college_enrollment['college_enroll'] = college_enrollment['TOT_ENROLL_p5']
college_enrollment.drop('TOT_ENROLL_p5', axis=1, inplace=True)

In [119]:
stationlevel_5mile = stationlevel_5mile.merge(college_enrollment, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,HH_below_50k_p5,hotelcount_p5,entertainmentcount_p5,restaurantcount_p5,intercityhub,prop_str_dens,Jobs_transit_p5,convenience_storecount_p5,highschoolenroll,college_enroll
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,922.4,NaN,NaN,NaN,NaN,87.782487,15074627.0,NaN,NaN,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,2932.3,NaN,NaN,4.0,40970.0,71.775767,47484669.0,3.0,1593.0,NaN
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,548.4,4.0,8.0,76.0,NaN,170.832426,13524248.0,1.0,728.0,NaN
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,0.0,NaN,NaN,NaN,27714.0,54.509764,0.0,NaN,NaN,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,193.1,NaN,NaN,3.0,NaN,5.291906,0.0,NaN,NaN,NaN


## High School Student Population based on Households

In [120]:
highschool_hh.head(2)

,Name_1,High_School_hh_p5
0,ADDISON ROAD-SEAT PLEASANT,177.0
1,ANACOSTIA,NaN


In [121]:
highschool_hh['High_School_hh'] = highschool_hh['High_School_hh_p5']
highschool_hh.drop('High_School_hh_p5', axis=1, inplace=True)

In [122]:
stationlevel_5mile = stationlevel_5mile.merge(highschool_hh, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,hotelcount_p5,entertainmentcount_p5,restaurantcount_p5,intercityhub,prop_str_dens,Jobs_transit_p5,convenience_storecount_p5,highschoolenroll,college_enroll,High_School_hh
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,NaN,NaN,NaN,NaN,87.782487,15074627.0,NaN,NaN,NaN,177.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,NaN,NaN,4.0,40970.0,71.775767,47484669.0,3.0,1593.0,NaN,NaN
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,4.0,8.0,76.0,NaN,170.832426,13524248.0,1.0,728.0,NaN,NaN
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,NaN,NaN,NaN,27714.0,54.509764,0.0,NaN,NaN,NaN,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,NaN,NaN,3.0,NaN,5.291906,0.0,NaN,NaN,NaN,NaN


## College Student Population based on Households

In [123]:
college_hh.head(2)

,Name_1,college_hh_p5
0,ADDISON ROAD-SEAT PLEASANT,32.0
1,ANACOSTIA,NaN


In [124]:
college_hh['college_hh'] = college_hh['college_hh_p5']
college_hh.drop('college_hh_p5', axis=1, inplace=True)

In [125]:
stationlevel_5mile = stationlevel_5mile.merge(college_hh, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,entertainmentcount_p5,restaurantcount_p5,intercityhub,prop_str_dens,Jobs_transit_p5,convenience_storecount_p5,highschoolenroll,college_enroll,High_School_hh,college_hh
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,NaN,NaN,NaN,87.782487,15074627.0,NaN,NaN,NaN,177.0,32.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,NaN,4.0,40970.0,71.775767,47484669.0,3.0,1593.0,NaN,NaN,NaN
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,8.0,76.0,NaN,170.832426,13524248.0,1.0,728.0,NaN,NaN,NaN
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,NaN,NaN,27714.0,54.509764,0.0,NaN,NaN,NaN,NaN,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,NaN,3.0,NaN,5.291906,0.0,NaN,NaN,NaN,NaN,NaN


## Household with zero cars

In [126]:
hh_0_car.head(2)

,Name_1,Num_of_Household_with_No_Car_p5
0,ADDISON ROAD-SEAT PLEASANT,16.0
1,ANACOSTIA,9.0


In [127]:
hh_0_car['HH_0_car'] = hh_0_car['Num_of_Household_with_No_Car_p5']
hh_0_car.drop('Num_of_Household_with_No_Car_p5', axis=1, inplace=True)

In [128]:
stationlevel_5mile = stationlevel_5mile.merge(hh_0_car, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,restaurantcount_p5,intercityhub,prop_str_dens,Jobs_transit_p5,convenience_storecount_p5,highschoolenroll,college_enroll,High_School_hh,college_hh,HH_0_car
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,NaN,NaN,87.782487,15074627.0,NaN,NaN,NaN,177.0,32.0,16.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,4.0,40970.0,71.775767,47484669.0,3.0,1593.0,NaN,NaN,NaN,9.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,76.0,NaN,170.832426,13524248.0,1.0,728.0,NaN,NaN,NaN,0.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,NaN,27714.0,54.509764,0.0,NaN,NaN,NaN,NaN,NaN,0.0
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,3.0,NaN,5.291906,0.0,NaN,NaN,NaN,NaN,NaN,0.0


## Households with exactly 1 car

In [129]:
hh_1_car.head(2)

,Name_1,Num_of_Household_with_1_Car_p5
0,ADDISON ROAD-SEAT PLEASANT,182.0
1,ANACOSTIA,62.5


In [130]:
hh_1_car['HH_1_car'] = hh_1_car['Num_of_Household_with_1_Car_p5']
hh_1_car.drop('Num_of_Household_with_1_Car_p5', axis=1, inplace=True)

In [131]:
stationlevel_5mile = stationlevel_5mile.merge(hh_1_car, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,intercityhub,prop_str_dens,Jobs_transit_p5,convenience_storecount_p5,highschoolenroll,college_enroll,High_School_hh,college_hh,HH_0_car,HH_1_car
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,NaN,87.782487,15074627.0,NaN,NaN,NaN,177.0,32.0,16.0,182.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,40970.0,71.775767,47484669.0,3.0,1593.0,NaN,NaN,NaN,9.0,62.5
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,NaN,170.832426,13524248.0,1.0,728.0,NaN,NaN,NaN,0.0,0.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,27714.0,54.509764,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,NaN,5.291906,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0


## Households with more than 1 car 

In [132]:
hh_morethan1_car.head(2)

,Name_1,Num_of_Household_with_More_than_One_Car_p5
0,ADDISON ROAD-SEAT PLEASANT,214.0
1,ANACOSTIA,14.5


In [133]:
hh_morethan1_car['HH_more1_car'] = hh_morethan1_car['Num_of_Household_with_More_than_One_Car_p5']
hh_morethan1_car.drop('Num_of_Household_with_More_than_One_Car_p5', axis=1, inplace=True)

In [134]:
stationlevel_5mile = stationlevel_5mile.merge(hh_morethan1_car, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,prop_str_dens,Jobs_transit_p5,convenience_storecount_p5,highschoolenroll,college_enroll,High_School_hh,college_hh,HH_0_car,HH_1_car,HH_more1_car
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,87.782487,15074627.0,NaN,NaN,NaN,177.0,32.0,16.0,182.0,214.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,71.775767,47484669.0,3.0,1593.0,NaN,NaN,NaN,9.0,62.5,14.5
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,170.832426,13524248.0,1.0,728.0,NaN,NaN,NaN,0.0,0.0,0.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,54.509764,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,5.291906,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


## Public Admin jobs

In [135]:
public_admin_jobs.head(2)

,Name_1,CNS20
0,ADDISON ROAD-SEAT PLEASANT,0.000000
1,ANACOSTIA,352.026486


In [136]:
public_admin_jobs['pub_admin_jobs'] = public_admin_jobs['CNS20']
public_admin_jobs.drop('CNS20', axis=1, inplace=True)

In [137]:
stationlevel_5mile = stationlevel_5mile.merge(public_admin_jobs, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,Jobs_transit_p5,convenience_storecount_p5,highschoolenroll,college_enroll,High_School_hh,college_hh,HH_0_car,HH_1_car,HH_more1_car,pub_admin_jobs
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,15074627.0,NaN,NaN,NaN,177.0,32.0,16.0,182.0,214.0,0.000000
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,47484669.0,3.0,1593.0,NaN,NaN,NaN,9.0,62.5,14.5,352.026486
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,13524248.0,1.0,728.0,NaN,NaN,NaN,0.0,0.0,0.0,0.000000
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000


## Number of Affordable Housing Units/Senior housing facilities within each walksheds

In [138]:
senior_housing_afford.head(2)

,Name_1,TotalUnits_p5
0,ANACOSTIA,1200
1,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,214


In [139]:
senior_housing_afford['housing_units_afford'] = senior_housing_afford['TotalUnits_p5']
senior_housing_afford.drop('TotalUnits_p5', axis=1, inplace=True)

In [140]:
stationlevel_5mile = stationlevel_5mile.merge(senior_housing_afford, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,convenience_storecount_p5,highschoolenroll,college_enroll,High_School_hh,college_hh,HH_0_car,HH_1_car,HH_more1_car,pub_admin_jobs,housing_units_afford
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,NaN,NaN,NaN,177.0,32.0,16.0,182.0,214.0,0.000000,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,3.0,1593.0,NaN,NaN,NaN,9.0,62.5,14.5,352.026486,1200.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,1.0,728.0,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,214.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,NaN


## SoP and Bike Lanes

In [141]:
sop.head(2)

,ID,Name_1,bikelane,form3norm,dens2norm,prox3norm,conn6norm,parks2norm,peds4norm,safenorm,traffic5norm,aesttot3norm,paf2norm,sop7norm
0,MSTN_062,ADDISON ROAD-SEAT PLEASANT,14.0,86.108428,57.880421,4.287417,74.808946,0.000000,30.254891,82.871252,50.478578,54.865260,0.000008,48.880712
1,MSTN_001,ANACOSTIA,9.0,80.626608,41.015569,11.695735,74.701052,4.361464,30.614129,77.408088,53.612582,50.533501,0.556780,47.302160


In [142]:
stationlevel_5mile = stationlevel_5mile.merge(sop, right_on = 'ID', left_on='ID', how = 'left')
stationlevel_5mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,dens2norm,prox3norm,conn6norm,parks2norm,peds4norm,safenorm,traffic5norm,aesttot3norm,paf2norm,sop7norm
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,57.880421,4.287417,74.808946,0.000000,30.254891,82.871252,50.478578,54.865260,0.000008,48.880712
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,41.015569,11.695735,74.701052,4.361464,30.614129,77.408088,53.612582,50.533501,0.556780,47.302160
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,62.777470,39.865198,71.479741,50.846523,40.163963,85.376394,61.637340,61.840846,3.617257,69.215061
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,21.897402,8.711603,72.984694,21.256899,36.410315,80.998596,47.971883,55.101324,0.000008,48.931768
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,30.287652,1.904513,73.178281,0.000000,19.069826,74.159322,43.160322,56.933369,0.000008,40.468556


In [143]:
stationlevel_5mile.shape

(98, 51)

# .75 Station Level Variables

## Load .75 Station Level Vars

In [144]:
households_half_mile    = pd.read_excel("../Data Preprocessing/Proportion of Households/output/walkshed_proportional_households_stations_p75.xlsx")
num_bus_lines           = pd.read_csv("../Data Preprocessing/Bus Lines and Stops/output/bus_line_stop_p75.csv")
jobs_half_mile          = pd.read_excel("../Data Preprocessing/Proportion of Jobs/output/proportional_jobs_stations_p75.xlsx")
nine_five_jobs          = pd.read_excel("../Data Preprocessing/Proportion of 9-5 Jobs/output/proportional_ninetofive_workers_stations_p75.xlsx", index_col=0)
school_jobs             = pd.read_excel("../Data Preprocessing/Proportion of Education Jobs/output/proportion_education_jobs_stations_p75.xlsx", index_col=0)
nite_wkend_jobs         = pd.read_excel("../Data Preprocessing/Proportion of Night and Weekend Jobs/output/Proportional_night_weekend_jobs_ML_p75.xlsx")
median_income           = pd.read_excel("../Data Preprocessing/Median Household Income/output/proportional_walkshed_household_income_updated_p75.xlsx", index_col=0)
hh_incomebelowpoverty   = pd.read_csv("../Data Preprocessing/HH_IncomeBelowPoverty/output/HH_Income_povertylevel_p75.csv")
hotels                  = pd.read_csv("../Data Preprocessing/Hotels Restaurants Entertainment/output/hotels_by_station_p75.csv").drop('Unnamed: 0', axis=1)
entertainment           = pd.read_csv("../Data Preprocessing/Hotels Restaurants Entertainment/output/entertainment_by_station_p75.csv").drop('Unnamed: 0', axis=1)
restaurant              = pd.read_csv("../Data Preprocessing/Hotels Restaurants Entertainment/output/restaurants_by_station_p75.csv").drop('Unnamed: 0', axis=1)
inter_city              = pd.read_csv("../Data Preprocessing/Intercity hubs/output/inter_cityhub_p75.csv")
intersection_density    = pd.read_csv("../Data Preprocessing/Intersection Density/output/proportion_density_walkstation_p75.csv")
job_access_by_transit   = pd.read_csv("../Data Preprocessing/Job Accessibility 45mins by transit and Auto/output/job_auto_and_transit_accessibity45mins_p75.csv")
convenience_store       = pd.read_csv("../Data Preprocessing/number of convenience stores/output/convenience_by_station_p75.csv").drop('Unnamed: 0', axis=1)
highschool_enrollment   = pd.read_csv("../Data Preprocessing/Number of education institutions/output/Enrollment_highschool_p75.csv")
college_enrollment      = pd.read_csv("../Data Preprocessing/Number of education institutions/output/Enrollment_colleges_p75.csv")
highschool_hh           = pd.read_csv("../Data Preprocessing/Number of Student Population/output/High_school_student_population_Enrollment_p75.csv")
college_hh              = pd.read_csv("../Data Preprocessing/Number of Student Population/output/grad_student_population_Enrollment_p75.csv")
hh_0_car                = pd.read_csv("../Data Preprocessing/Number of Households with no car or one car/output/victor-no of household w no car_p75.csv")
hh_1_car                = pd.read_csv("../Data Preprocessing/Number of Households with no car or one car/output/victor-no of household w 1 car_p75.csv")
hh_morethan1_car        = pd.read_csv("../Data Preprocessing/Number of Households with no car or one car/output/victor-no of household w more than 1cars_p75.csv")
public_admin_jobs       = pd.read_csv("../Data Preprocessing/Public Admin Jobs/output/proportional_publicadmin_workers_stations_p75.csv")
senior_housing_afford   = pd.read_csv("../Data Preprocessing/seniorhousing_affordability/output/senior_housing_facilitiesDMV_p75.csv").drop('Unnamed: 0', axis=1)
bikeshare_cap           = pd.read_csv("../Data Preprocessing/bikeshare capacity/output/bikeshare_capacity_p75.csv")
ctpp_jobs               = pd.read_csv("../Data Preprocessing/CTPP Jobs Data/output/CTPP_jobs_p75.csv")
sop                     = pd.read_csv("../Data Preprocessing/SoP Data/output/SoP_BikeLanes_p75.csv", index_col = 0)

## AM Trains per Hour

* Filter only for weekday services and in the AM peak
* Groupby and get the sum of trains per hour
* Merge into the main AM dataframe

In [145]:
amtrains_per_hour2= trains_per_hour[(trains_per_hour['ARRIVAL_PERIOD']=='AM Peak') & (trains_per_hour['SERVICETYPE']=='Weekday')]
amtrains_per_hour2.head()

,STATION_ID,STOP_ID,DIR_ID,SERVICETYPE,ARRIVAL_PERIOD,AVG_TRAINS
8,MSTN_001,F06,1,Weekday,AM Peak,17
10,MSTN_001,F06,2,Weekday,AM Peak,16
20,MSTN_002,F02,1,Weekday,AM Peak,31
22,MSTN_002,F02,2,Weekday,AM Peak,31
32,MSTN_003,G01,1,Weekday,AM Peak,24


In [146]:
amtrains_per_hour2 = amtrains_per_hour2.groupby(['STATION_ID']).sum().reset_index()
amtrains_per_hour2 = amtrains_per_hour2[['STATION_ID','AVG_TRAINS']]
amtrains_per_hour2.rename(columns = {'AVG_TRAINS': 'AM_AVG_TRAINS'}, inplace=True)
amtrains_per_hour2.head()

,STATION_ID,AM_AVG_TRAINS
0,MSTN_001,33
1,MSTN_002,62
2,MSTN_003,51
3,MSTN_004,56
4,MSTN_005,78


In [147]:
stationlevel_75mile = mstns.merge(amtrains_per_hour2, left_on = 'ID', right_on = 'STATION_ID', how = 'left').drop(['STATION_ID','PRIMARY_NAME_FY23', 'walkshed_filename', 'WMATA_filename', 'track_miles_names', 'auto_filename'	 ], axis =1)
stationlevel_75mile

,ID,AM_AVG_TRAINS
0,MSTN_062,53.0
1,MSTN_001,33.0
2,MSTN_002,62.0
3,MSTN_065,27.0
4,MSTN_097,NaN
...,...,...
93,MSTN_059,28.0
94,MSTN_055,62.0
95,MSTN_046,56.0
96,MSTN_091,27.0


## PM Trains per Hour

* Filter only for weekday services and in the PM peak
* Groupby and get the sum of trains per hour
* Merge into the PM dataframe first based on the origins and again based on the destinations
* Drop redundant columns

In [148]:
pmtrains_per_hour2 = trains_per_hour[(trains_per_hour['ARRIVAL_PERIOD']=='PM Peak') & (trains_per_hour['SERVICETYPE']=='Weekday')]
pmtrains_per_hour2.head()

,STATION_ID,STOP_ID,DIR_ID,SERVICETYPE,ARRIVAL_PERIOD,AVG_TRAINS
9,MSTN_001,F06,1,Weekday,PM Peak,17
11,MSTN_001,F06,2,Weekday,PM Peak,17
21,MSTN_002,F02,1,Weekday,PM Peak,31
23,MSTN_002,F02,2,Weekday,PM Peak,31
33,MSTN_003,G01,1,Weekday,PM Peak,24


In [149]:
pmtrains_per_hour2 = pmtrains_per_hour2.groupby(['STATION_ID']).sum().reset_index()
pmtrains_per_hour2 = pmtrains_per_hour2[['STATION_ID','AVG_TRAINS']]
pmtrains_per_hour2.rename(columns = {'AVG_TRAINS': 'PM_AVG_TRAINS'}, inplace = True)
pmtrains_per_hour2.head()

,STATION_ID,PM_AVG_TRAINS
0,MSTN_001,34
1,MSTN_002,62
2,MSTN_003,48
3,MSTN_004,53
4,MSTN_005,74


In [150]:
stationlevel_75mile = stationlevel_75mile.merge(pmtrains_per_hour2, left_on = 'ID', right_on='STATION_ID', how = 'left')

In [151]:
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS
0,MSTN_062,53.0,MSTN_062,48.0
1,MSTN_001,33.0,MSTN_001,34.0
2,MSTN_002,62.0,MSTN_002,62.0
3,MSTN_065,27.0,MSTN_065,24.0
4,MSTN_097,NaN,NaN,NaN


## All Periods, Trains per Hour

In [152]:
all_trains_per_hour.head()

,ID,ALLPERIODS_AVG_TRAINS
0,MSTN_001,3.400000
1,MSTN_002,6.340909
2,MSTN_003,5.318182
3,MSTN_004,5.311111
4,MSTN_005,8.186047


In [153]:
all_trains_per_hour.rename(columns = {'STATION_ID': 'ID'}, inplace = True)

In [154]:
stationlevel_75mile = stationlevel_75mile.merge(all_trains_per_hour, on='ID', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS
0,MSTN_062,53.0,MSTN_062,48.0,5.340909
1,MSTN_001,33.0,MSTN_001,34.0,3.400000
2,MSTN_002,62.0,MSTN_002,62.0,6.340909
3,MSTN_065,27.0,MSTN_065,24.0,2.904762
4,MSTN_097,NaN,NaN,NaN,NaN


## Households per Half Mile

* Subset only the ID, proportion of houses and the total number of households. Rename ID to MSTN
* Merge once based on the destinations and again based on the origins
* Drop duplicate columns

In [155]:
households_half_mile = households_half_mile[['ID', 'proportionhouses', 'Total Households', ]]
households_half_mile.rename(columns={'ID':'MSTN'}, inplace=True)

In [156]:
households_half_mile.head()

,MSTN,proportionhouses,Total Households
0,MSTN_062,3.042234,4450
1,MSTN_001,5.065567,6579
2,MSTN_002,9.904623,12975
3,MSTN_065,2.142590,3154
4,MSTN_097,2.967461,4375


In [157]:
stationlevel_75mile = stationlevel_75mile.merge(households_half_mile, right_on = 'MSTN', left_on='ID', how = 'left').drop(['MSTN'], axis = 1)
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0


## Parking Capacity

* Drop the station column from parking_capacity and rename the MSTN_ID column to avoid errors while merging
* Merge into the AM dataframe first based on the origins and again based on the destinations
* Fill NAs in parking capacity with 0

In [158]:
stationlevel_75mile = stationlevel_75mile.merge(parking_capacity, right_on = 'MSTN_ID', left_on='ID', how = 'left').drop(['MSTN_ID'], axis = 1)
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0


## Number of Bus Lines

* Rename the bus line count and bus stop columns to distinguish them
* Merge into the main AM dataframe, first based on destinations and again based on origins
* Drop columns that are redundant

In [159]:
num_bus_lines.head()

,Name_1,MSTN,line_count_p75,stop_count_p75
0,ADDISON ROAD-SEAT PLEASANT,MSTN_062,36.0,33.0
1,ANACOSTIA,MSTN_001,69.0,75.0
2,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,MSTN_002,137.0,145.0
3,ARLINGTON CEMETERY,MSTN_065,17.0,1.0
4,BALLSTON-MU,MSTN_068,37.0,59.0


In [160]:
num_bus_lines.rename(columns={'line_count_p75':'bus_line_count','stop_count_p75':'bus_stop_count'}, inplace=True)
num_bus_lines = num_bus_lines[['MSTN','bus_line_count','bus_stop_count']]

In [161]:
stationlevel_75mile = stationlevel_75mile.merge(num_bus_lines, right_on = 'MSTN', left_on='ID', how = 'left').drop(['MSTN'], axis = 1)
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0


## Jobs per Half Mile

* Merge Jobs into the AM dataframe first based on the origins and again based on the destinations

In [162]:
jobs_half_mile2 = mstns.merge(jobs_half_mile, left_on = 'walkshed_filename', right_on='Name_1').drop(['PRIMARY_NAME_FY23', 'walkshed_filename', 'WMATA_filename',
       'track_miles_names', 'auto_filename', 'Name_1'], axis = 1)

In [163]:
stationlevel_75mile = stationlevel_75mile.merge(jobs_half_mile2, right_on = 'ID', left_on='ID', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p75
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,2467.379609
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,8900.092200
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,82346.160940
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,18636.805827
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,5333.381005


## Distance to the Core

* Give the station names in dist_to_core proper formatting to merge with mstns
* Give station names proper MSTN IDs
* Subset only the ID and distance_miles columns. Rename both for easy merging and clarity
* Merge into the AM dataframe first based on destinations, then based on origins

In [164]:
dist_to_core['STATIONNAM'] = dist_to_core['STATIONNAM'].str.replace(' : 0 - 2640','')
dist_to_core['STATIONNAM'] = dist_to_core['STATIONNAM'].str.replace(' : 0 - 22.4525758392805','')
dist_to_core['STATIONNAM'] = dist_to_core['STATIONNAM'].str.title()
dist_to_core['STATIONNAM'] = dist_to_core['STATIONNAM'].replace({"Addison Road-Seat Pleasant": 'Addison Road', "Archives-Navy Mem'L-Penn Quarter":"Archives", "Ballston-Mu": "Ballston-MU", "Brookland-Cua":"Brookland-CUA", "Dunn Loring-Merrifield": "Dunn Loring", "Largo Town Center":"Downtown Largo", "Eisenhower Avenue":"Eisenhower Ave", 'Federal Center Sw':'Federal Center SW', "Foggy Bottom-Gwu":"Foggy Bottom-GWU","Gallery Pl-Chinatown":"Gallary Place", "Mclean":"McLean", "Mcpherson Square":"McPherson Sq", "Mt Vernon Sq/7Th St-Convention Center":"Mt Vernon Sq", "Noma-Gallaudet U-New York Ave":"NoMa-Gallaudet U", "Rhode Island Ave-Brentwood":"Rhode Island Ave",  "Prince George'S Plaza":"Hyattsville Crossing", "Tenleytown-Au": "Tenleytown-AU", "Tysons Corner":"Tysons", "U St/African-Amer Civil War Memorial/Cardozo": "U Street", "Southern Avenue": "Southern Ave", "Van Ness-Udc":"Van Ness-UDC", "Virginia Square-Gmu":"Virginia Sq-GMU", "Vienna/Fairfax-Gmu":"Vienna", "Washington Dulles International Airport":"Dulles Airport", "West Falls Church-Vt/Uva":"West Falls Church", "Woodley Park-Zoo/Adams Morgan":"Woodley Park", "White Flint":"North Bethesda" })
dist_to_core['STATIONNAM'] = dist_to_core['STATIONNAM'].replace({"College Park-U Of Md": 'College Park-U of Md', "Gallary Place": "Gallery Place", "Shaw-Howard Univ":"Shaw-Howard U", "Innovation":"Innovation Center", "Reston":"Reston Town Center"})

In [165]:
dist_to_core2 = mstns.merge(dist_to_core, left_on = 'PRIMARY_NAME_FY23', right_on= 'STATIONNAM')

In [166]:
dist_to_core2 = dist_to_core2[['ID','distance_miles']]
dist_to_core2.rename(columns={'ID':'id1', 'distance_miles':'distance_to_core'}, inplace=True)

In [167]:
stationlevel_75mile = stationlevel_75mile.merge(dist_to_core2, right_on = 'id1', left_on='ID', how = 'left').drop(['id1'], axis = 1)
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p75,distance_to_core
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,2467.379609,7.181143
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,8900.092200,3.007895
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,82346.160940,0.435964
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,18636.805827,2.127338
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,5333.381005,25.994746


## Night and Weekend Jobs

* Merge into the Off Peak dataframe first based on the origins and again based on the destinations

In [168]:
nite_wkend_jobs.head()

,Name_1,Proportion_night_weekend_jobs_p75
0,ADDISON ROAD-SEAT PLEASANT,398.822135
1,ANACOSTIA,410.559374
2,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,4301.547105
3,ARLINGTON CEMETERY,749.475643
4,Ashburn,1533.058436


In [169]:
nite_wkend_jobs2 = mstns.merge(nite_wkend_jobs, left_on = 'walkshed_filename', right_on= 'Name_1')
nite_wkend_jobs2 = nite_wkend_jobs2[['ID', 'Name_1', 'Proportion_night_weekend_jobs_p75']]

In [170]:
stationlevel_75mile = stationlevel_75mile.merge(nite_wkend_jobs2, on='ID', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p75,distance_to_core,Name_1,Proportion_night_weekend_jobs_p75
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,2467.379609,7.181143,ADDISON ROAD-SEAT PLEASANT,398.822135
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,8900.092200,3.007895,ANACOSTIA,410.559374
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,82346.160940,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,4301.547105
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,18636.805827,2.127338,ARLINGTON CEMETERY,749.475643
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,5333.381005,25.994746,Ashburn,1533.058436


## 9-5 Jobs

In [171]:
nine_five_jobs

,Name_1,Total_Nine_to_Five_workers_p75
0,ADDISON ROAD-SEAT PLEASANT,122.305455
1,ANACOSTIA,3862.041572
2,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,27820.888482
3,ARLINGTON CEMETERY,17405.524413
4,Ashburn,2338.837644
...,...,...
92,WHEATON,843.578203
93,WHITE FLINT,7537.868260
94,WIEHLE-RESTON EAST,300.750951
95,WOODLEY PARK-ZOO/ADAMS MORGAN,2229.542839


In [172]:
stationlevel_75mile = stationlevel_75mile.merge(nine_five_jobs, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p75,distance_to_core,Name_1,Proportion_night_weekend_jobs_p75,Total_Nine_to_Five_workers_p75
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,2467.379609,7.181143,ADDISON ROAD-SEAT PLEASANT,398.822135,122.305455
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,8900.092200,3.007895,ANACOSTIA,410.559374,3862.041572
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,82346.160940,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,4301.547105,27820.888482
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,18636.805827,2.127338,ARLINGTON CEMETERY,749.475643,17405.524413
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,5333.381005,25.994746,Ashburn,1533.058436,2338.837644


## School Jobs

In [173]:
school_jobs.reset_index(inplace = True)

In [174]:
stationlevel_75mile = stationlevel_75mile.merge(school_jobs, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p75,distance_to_core,Name_1,Proportion_night_weekend_jobs_p75,Total_Nine_to_Five_workers_p75,Proportion_education_jobs_p75
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,2467.379609,7.181143,ADDISON ROAD-SEAT PLEASANT,398.822135,122.305455,1116.701978
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,8900.092200,3.007895,ANACOSTIA,410.559374,3862.041572,607.720656
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,82346.160940,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,4301.547105,27820.888482,670.535284
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,18636.805827,2.127338,ARLINGTON CEMETERY,749.475643,17405.524413,2.327564
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,5333.381005,25.994746,Ashburn,1533.058436,2338.837644,713.426290


## Median Household Income

* Rename the station name column to make it easier for merging 
* Merge into the Off Peak dataframe first based on the origins and again based on the destinations
* Fill any NAs with 0

In [175]:
stationlevel_75mile = stationlevel_75mile.merge(median_income, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p75,distance_to_core,Name_1,Proportion_night_weekend_jobs_p75,Total_Nine_to_Five_workers_p75,Proportion_education_jobs_p75,Median_household_income_p75
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,2467.379609,7.181143,ADDISON ROAD-SEAT PLEASANT,398.822135,122.305455,1116.701978,77702.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,8900.092200,3.007895,ANACOSTIA,410.559374,3862.041572,607.720656,32458.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,82346.160940,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,4301.547105,27820.888482,670.535284,135011.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,18636.805827,2.127338,ARLINGTON CEMETERY,749.475643,17405.524413,2.327564,125578.5
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,5333.381005,25.994746,Ashburn,1533.058436,2338.837644,713.426290,133298.0


## Terminal Station Dummy

* Subset only the dummy column and the station IDs
* Merge into the AM dataframe once based on destinations and again based on origins

In [176]:
terminal_station2 = terminal_station[['MSTN','terminal_dummy_2023']]

In [177]:
stationlevel_75mile = stationlevel_75mile.merge(terminal_station2, right_on = 'MSTN', left_on='ID', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p75,distance_to_core,Name_1,Proportion_night_weekend_jobs_p75,Total_Nine_to_Five_workers_p75,Proportion_education_jobs_p75,Median_household_income_p75,MSTN,terminal_dummy_2023
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,2467.379609,7.181143,ADDISON ROAD-SEAT PLEASANT,398.822135,122.305455,1116.701978,77702.0,MSTN_062,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,8900.092200,3.007895,ANACOSTIA,410.559374,3862.041572,607.720656,32458.0,MSTN_001,NaN
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,82346.160940,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,4301.547105,27820.888482,670.535284,135011.0,MSTN_002,NaN
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,18636.805827,2.127338,ARLINGTON CEMETERY,749.475643,17405.524413,2.327564,125578.5,MSTN_065,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,5333.381005,25.994746,Ashburn,1533.058436,2338.837644,713.426290,133298.0,MSTN_097,1.0


## Bikeshare Capacity

In [178]:
bikeshare_cap.head()

,Name_1,CAPACITY_p75
0,ANACOSTIA,71
1,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,988
2,ARLINGTON CEMETERY,19
3,BALLSTON-MU,216
4,BENNING ROAD,56


In [179]:
bikeshare_cap['bike_cap'] = bikeshare_cap['CAPACITY_p75']
bikeshare_cap = bikeshare_cap.drop('CAPACITY_p75', axis=1)

In [180]:
stationlevel_75mile = stationlevel_75mile.merge(bikeshare_cap, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,All_Jobs_p75,distance_to_core,Name_1,Proportion_night_weekend_jobs_p75,Total_Nine_to_Five_workers_p75,Proportion_education_jobs_p75,Median_household_income_p75,MSTN,terminal_dummy_2023,bike_cap
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,2467.379609,7.181143,ADDISON ROAD-SEAT PLEASANT,398.822135,122.305455,1116.701978,77702.0,MSTN_062,NaN,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,8900.092200,3.007895,ANACOSTIA,410.559374,3862.041572,607.720656,32458.0,MSTN_001,NaN,71.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,82346.160940,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,4301.547105,27820.888482,670.535284,135011.0,MSTN_002,NaN,988.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,18636.805827,2.127338,ARLINGTON CEMETERY,749.475643,17405.524413,2.327564,125578.5,MSTN_065,NaN,19.0
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,5333.381005,25.994746,Ashburn,1533.058436,2338.837644,713.426290,133298.0,MSTN_097,1.0,NaN


## CTPP Jobs

In [181]:
ctpp_jobs.head()

,ID,PRIMARY_NAME_FY23,walkshed_filename,Name_1,Workers 16 and Over_p75,proportionjobs_p75
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,ADDISON ROAD-SEAT PLEASANT,1560,680.109494
1,MSTN_001,Anacostia,ANACOSTIA,ANACOSTIA,19805,7254.574071
2,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,234605,120332.203372
3,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,ARLINGTON CEMETERY,34175,12522.801407
4,MSTN_097,Ashburn,Ashburn,Ashburn,5205,1175.475981


In [182]:
# pd.set_option('display.max_rows', None)
# pd.options.display.max_rows = None
# pd.reset_option('all')
ctpp_jobs = ctpp_jobs[['Name_1', 'proportionjobs_p75' ]].rename(columns={'proportionjobs_p75': 'ctpp_jobs'}).copy()

In [183]:
ctpp_jobs.head(2)

,Name_1,ctpp_jobs
0,ADDISON ROAD-SEAT PLEASANT,680.109494
1,ANACOSTIA,7254.574071


In [184]:
stationlevel_75mile = stationlevel_75mile.merge(ctpp_jobs, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,distance_to_core,Name_1,Proportion_night_weekend_jobs_p75,Total_Nine_to_Five_workers_p75,Proportion_education_jobs_p75,Median_household_income_p75,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,7.181143,ADDISON ROAD-SEAT PLEASANT,398.822135,122.305455,1116.701978,77702.0,MSTN_062,NaN,NaN,680.109494
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,3.007895,ANACOSTIA,410.559374,3862.041572,607.720656,32458.0,MSTN_001,NaN,71.0,7254.574071
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,0.435964,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,4301.547105,27820.888482,670.535284,135011.0,MSTN_002,NaN,988.0,120332.203372
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,2.127338,ARLINGTON CEMETERY,749.475643,17405.524413,2.327564,125578.5,MSTN_065,NaN,19.0,12522.801407
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,25.994746,Ashburn,1533.058436,2338.837644,713.426290,133298.0,MSTN_097,1.0,NaN,1175.475981


## Household Income below 50k

In [185]:
hh_incomebelowpoverty.head(2)

,Name_1,HH_below_50k_p75
0,ADDISON ROAD-SEAT PLEASANT,2950.8
1,ANACOSTIA,5447.9


In [186]:
stationlevel_75mile = stationlevel_75mile.merge(hh_incomebelowpoverty, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,Name_1,Proportion_night_weekend_jobs_p75,Total_Nine_to_Five_workers_p75,Proportion_education_jobs_p75,Median_household_income_p75,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p75
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,ADDISON ROAD-SEAT PLEASANT,398.822135,122.305455,1116.701978,77702.0,MSTN_062,NaN,NaN,680.109494,2950.8
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,ANACOSTIA,410.559374,3862.041572,607.720656,32458.0,MSTN_001,NaN,71.0,7254.574071,5447.9
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,4301.547105,27820.888482,670.535284,135011.0,MSTN_002,NaN,988.0,120332.203372,2587.6
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,ARLINGTON CEMETERY,749.475643,17405.524413,2.327564,125578.5,MSTN_065,NaN,19.0,12522.801407,208.5
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,Ashburn,1533.058436,2338.837644,713.426290,133298.0,MSTN_097,1.0,NaN,1175.475981,773.9


## Number of hotels within the walkshed

In [187]:
hotels.head(2)

,Name_1,hotelcount_p75
0,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,14
1,BETHESDA,1


In [188]:
stationlevel_75mile = stationlevel_75mile.merge(hotels, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,Proportion_night_weekend_jobs_p75,Total_Nine_to_Five_workers_p75,Proportion_education_jobs_p75,Median_household_income_p75,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p75,hotelcount_p75
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,398.822135,122.305455,1116.701978,77702.0,MSTN_062,NaN,NaN,680.109494,2950.8,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,410.559374,3862.041572,607.720656,32458.0,MSTN_001,NaN,71.0,7254.574071,5447.9,NaN
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,4301.547105,27820.888482,670.535284,135011.0,MSTN_002,NaN,988.0,120332.203372,2587.6,14.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,749.475643,17405.524413,2.327564,125578.5,MSTN_065,NaN,19.0,12522.801407,208.5,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,1533.058436,2338.837644,713.426290,133298.0,MSTN_097,1.0,NaN,1175.475981,773.9,NaN


## Number of entertainment locations within the walkshed

In [189]:
entertainment.head()

,Name_1,entertainmentcount_p75
0,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,14
1,BALLSTON-MU,1
2,BENNING ROAD,1
3,BETHESDA,4
4,BRADDOCK ROAD,1


In [190]:
stationlevel_75mile = stationlevel_75mile.merge(entertainment, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,Total_Nine_to_Five_workers_p75,Proportion_education_jobs_p75,Median_household_income_p75,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p75,hotelcount_p75,entertainmentcount_p75
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,122.305455,1116.701978,77702.0,MSTN_062,NaN,NaN,680.109494,2950.8,NaN,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,3862.041572,607.720656,32458.0,MSTN_001,NaN,71.0,7254.574071,5447.9,NaN,NaN
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,27820.888482,670.535284,135011.0,MSTN_002,NaN,988.0,120332.203372,2587.6,14.0,14.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,17405.524413,2.327564,125578.5,MSTN_065,NaN,19.0,12522.801407,208.5,NaN,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,2338.837644,713.426290,133298.0,MSTN_097,1.0,NaN,1175.475981,773.9,NaN,NaN


## Number of restaurants located within each walksheds

In [191]:
restaurant.head(2)

,Name_1,restaurantcount_p75
0,ANACOSTIA,8
1,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,157


In [192]:
stationlevel_75mile = stationlevel_75mile.merge(restaurant, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,Proportion_education_jobs_p75,Median_household_income_p75,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p75,hotelcount_p75,entertainmentcount_p75,restaurantcount_p75
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,1116.701978,77702.0,MSTN_062,NaN,NaN,680.109494,2950.8,NaN,NaN,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,607.720656,32458.0,MSTN_001,NaN,71.0,7254.574071,5447.9,NaN,NaN,8.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,670.535284,135011.0,MSTN_002,NaN,988.0,120332.203372,2587.6,14.0,14.0,157.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,2.327564,125578.5,MSTN_065,NaN,19.0,12522.801407,208.5,NaN,NaN,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,713.426290,133298.0,MSTN_097,1.0,NaN,1175.475981,773.9,NaN,NaN,6.0


## Station is an Inter-city hub (by ridership)

In [193]:
inter_city.head(2)

,Name_1,ridership_p75
0,ADDISON ROAD,19259.0
1,ANACOSTIA,40970.0


In [194]:
inter_city['intercityhub'] = inter_city['ridership_p75']
inter_city.drop('ridership_p75', axis=1, inplace=True)

In [195]:
stationlevel_75mile = stationlevel_75mile.merge(inter_city, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,Median_household_income_p75,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p75,hotelcount_p75,entertainmentcount_p75,restaurantcount_p75,intercityhub
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,77702.0,MSTN_062,NaN,NaN,680.109494,2950.8,NaN,NaN,NaN,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,32458.0,MSTN_001,NaN,71.0,7254.574071,5447.9,NaN,NaN,8.0,40970.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,135011.0,MSTN_002,NaN,988.0,120332.203372,2587.6,14.0,14.0,157.0,NaN
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,125578.5,MSTN_065,NaN,19.0,12522.801407,208.5,NaN,NaN,NaN,27714.0
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,133298.0,MSTN_097,1.0,NaN,1175.475981,773.9,NaN,NaN,6.0,NaN


## Intersection density

In [196]:
intersection_density.head(2)

,Name_1,proportional_street_density_p75
0,ADDISON ROAD-SEAT PLEASANT,485.968261
1,ANACOSTIA,383.409456


In [197]:
intersection_density['prop_str_dens'] = intersection_density['proportional_street_density_p75']
intersection_density.drop('proportional_street_density_p75', axis=1, inplace=True)

In [198]:
stationlevel_75mile = stationlevel_75mile.merge(intersection_density, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,MSTN,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p75,hotelcount_p75,entertainmentcount_p75,restaurantcount_p75,intercityhub,prop_str_dens
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,MSTN_062,NaN,NaN,680.109494,2950.8,NaN,NaN,NaN,NaN,485.968261
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,MSTN_001,NaN,71.0,7254.574071,5447.9,NaN,NaN,8.0,40970.0,383.409456
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,MSTN_002,NaN,988.0,120332.203372,2587.6,14.0,14.0,157.0,NaN,933.339602
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,MSTN_065,NaN,19.0,12522.801407,208.5,NaN,NaN,NaN,27714.0,513.617499
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,MSTN_097,1.0,NaN,1175.475981,773.9,NaN,NaN,6.0,NaN,149.543351


## Job Accessibility by Transit

In [199]:
job_access_by_transit.head(2)

,Name_1,Jobs_auto,Jobs_transit_p75
0,ADDISON ROAD-SEAT PLEASANT,82012248.0,53581136.0
1,ANACOSTIA,97007534.0,103249729.0


In [200]:
job_access_by_transit = job_access_by_transit[['Name_1', 'Jobs_transit_p75']]

In [201]:
stationlevel_75mile = stationlevel_75mile.merge(job_access_by_transit, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,terminal_dummy_2023,bike_cap,ctpp_jobs,HH_below_50k_p75,hotelcount_p75,entertainmentcount_p75,restaurantcount_p75,intercityhub,prop_str_dens,Jobs_transit_p75
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,NaN,NaN,680.109494,2950.8,NaN,NaN,NaN,NaN,485.968261,53581136.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,NaN,71.0,7254.574071,5447.9,NaN,NaN,8.0,40970.0,383.409456,103249729.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,NaN,988.0,120332.203372,2587.6,14.0,14.0,157.0,NaN,933.339602,108026328.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,NaN,19.0,12522.801407,208.5,NaN,NaN,NaN,27714.0,513.617499,12676878.0
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,1.0,NaN,1175.475981,773.9,NaN,NaN,6.0,NaN,149.543351,0.0


## Number of convenience stores within each walksheds

In [202]:
convenience_store.head(2)

,Name_1,convenience_storecount_p75
0,ANACOSTIA,7
1,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,8


In [203]:
stationlevel_75mile = stationlevel_75mile.merge(convenience_store, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,bike_cap,ctpp_jobs,HH_below_50k_p75,hotelcount_p75,entertainmentcount_p75,restaurantcount_p75,intercityhub,prop_str_dens,Jobs_transit_p75,convenience_storecount_p75
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,NaN,680.109494,2950.8,NaN,NaN,NaN,NaN,485.968261,53581136.0,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,71.0,7254.574071,5447.9,NaN,NaN,8.0,40970.0,383.409456,103249729.0,7.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,988.0,120332.203372,2587.6,14.0,14.0,157.0,NaN,933.339602,108026328.0,8.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,19.0,12522.801407,208.5,NaN,NaN,NaN,27714.0,513.617499,12676878.0,1.0
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,NaN,1175.475981,773.9,NaN,NaN,6.0,NaN,149.543351,0.0,NaN


## High School Enrollment

In [204]:
highschool_enrollment.head(2)

,Name_1,TOTAL_p75
0,ADDISON ROAD-SEAT PLEASANT,777.0
1,ANACOSTIA,3634.0


In [205]:
highschool_enrollment['highschoolenroll'] = highschool_enrollment['TOTAL_p75']
highschool_enrollment.drop('TOTAL_p75', axis=1, inplace=True)

In [206]:
stationlevel_75mile = stationlevel_75mile.merge(highschool_enrollment, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,ctpp_jobs,HH_below_50k_p75,hotelcount_p75,entertainmentcount_p75,restaurantcount_p75,intercityhub,prop_str_dens,Jobs_transit_p75,convenience_storecount_p75,highschoolenroll
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,680.109494,2950.8,NaN,NaN,NaN,NaN,485.968261,53581136.0,NaN,777.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,7254.574071,5447.9,NaN,NaN,8.0,40970.0,383.409456,103249729.0,7.0,3634.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,120332.203372,2587.6,14.0,14.0,157.0,NaN,933.339602,108026328.0,8.0,1616.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,12522.801407,208.5,NaN,NaN,NaN,27714.0,513.617499,12676878.0,1.0,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,1175.475981,773.9,NaN,NaN,6.0,NaN,149.543351,0.0,NaN,NaN


## College Enrollment within the walkshed

In [207]:
college_enrollment.head()

,Name_1,TOT_ENROLL_p75
0,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,232.0
1,BALLSTON-MU,8.0
2,BETHESDA,1071.0
3,BRANCH AVE,2760.0
4,BROOKLAND-CUA,7636.0


In [208]:
college_enrollment['college_enroll'] = college_enrollment['TOT_ENROLL_p75']
college_enrollment.drop('TOT_ENROLL_p75', axis=1, inplace=True)

In [209]:
stationlevel_75mile = stationlevel_75mile.merge(college_enrollment, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,HH_below_50k_p75,hotelcount_p75,entertainmentcount_p75,restaurantcount_p75,intercityhub,prop_str_dens,Jobs_transit_p75,convenience_storecount_p75,highschoolenroll,college_enroll
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,2950.8,NaN,NaN,NaN,NaN,485.968261,53581136.0,NaN,777.0,NaN
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,5447.9,NaN,NaN,8.0,40970.0,383.409456,103249729.0,7.0,3634.0,NaN
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,2587.6,14.0,14.0,157.0,NaN,933.339602,108026328.0,8.0,1616.0,232.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,208.5,NaN,NaN,NaN,27714.0,513.617499,12676878.0,1.0,NaN,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,773.9,NaN,NaN,6.0,NaN,149.543351,0.0,NaN,NaN,NaN


## College Student Population based on Households

In [210]:
college_hh.head(2)

,Name_1,college_hh_p75
0,ADDISON ROAD-SEAT PLEASANT,32.0
1,ANACOSTIA,NaN


In [211]:
college_hh['college_hh'] = college_hh['college_hh_p75']
college_hh.drop('college_hh_p75', axis=1, inplace=True)

In [212]:
stationlevel_75mile = stationlevel_75mile.merge(college_hh, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,hotelcount_p75,entertainmentcount_p75,restaurantcount_p75,intercityhub,prop_str_dens,Jobs_transit_p75,convenience_storecount_p75,highschoolenroll,college_enroll,college_hh
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,NaN,NaN,NaN,NaN,485.968261,53581136.0,NaN,777.0,NaN,32.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,NaN,NaN,8.0,40970.0,383.409456,103249729.0,7.0,3634.0,NaN,NaN
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,14.0,14.0,157.0,NaN,933.339602,108026328.0,8.0,1616.0,232.0,NaN
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,NaN,NaN,NaN,27714.0,513.617499,12676878.0,1.0,NaN,NaN,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,NaN,NaN,6.0,NaN,149.543351,0.0,NaN,NaN,NaN,NaN


## High School Student Population based on Households

In [213]:
highschool_hh.head(2)

,Name_1,High_School_hh_p75
0,ADDISON ROAD-SEAT PLEASANT,177.0
1,ANACOSTIA,NaN


In [214]:
highschool_hh['High_School_hh'] = highschool_hh['High_School_hh_p75']
highschool_hh.drop('High_School_hh_p75', axis=1, inplace=True)

In [215]:
stationlevel_75mile = stationlevel_75mile.merge(highschool_hh, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,entertainmentcount_p75,restaurantcount_p75,intercityhub,prop_str_dens,Jobs_transit_p75,convenience_storecount_p75,highschoolenroll,college_enroll,college_hh,High_School_hh
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,NaN,NaN,NaN,485.968261,53581136.0,NaN,777.0,NaN,32.0,177.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,NaN,8.0,40970.0,383.409456,103249729.0,7.0,3634.0,NaN,NaN,NaN
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,14.0,157.0,NaN,933.339602,108026328.0,8.0,1616.0,232.0,NaN,NaN
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,NaN,NaN,27714.0,513.617499,12676878.0,1.0,NaN,NaN,NaN,NaN
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,NaN,6.0,NaN,149.543351,0.0,NaN,NaN,NaN,NaN,NaN


## Household with zero cars

In [216]:
hh_0_car.head(2)

,Name_1,Num_of_Household_with_No_Car_p75
0,ADDISON ROAD-SEAT PLEASANT,16.0
1,ANACOSTIA,8.0


In [217]:
hh_0_car['HH_0_car'] = hh_0_car['Num_of_Household_with_No_Car_p75']
hh_0_car.drop('Num_of_Household_with_No_Car_p75', axis=1, inplace=True)

In [218]:
stationlevel_75mile = stationlevel_75mile.merge(hh_0_car, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,restaurantcount_p75,intercityhub,prop_str_dens,Jobs_transit_p75,convenience_storecount_p75,highschoolenroll,college_enroll,college_hh,High_School_hh,HH_0_car
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,NaN,NaN,485.968261,53581136.0,NaN,777.0,NaN,32.0,177.0,16.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,8.0,40970.0,383.409456,103249729.0,7.0,3634.0,NaN,NaN,NaN,8.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,157.0,NaN,933.339602,108026328.0,8.0,1616.0,232.0,NaN,NaN,10.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,NaN,27714.0,513.617499,12676878.0,1.0,NaN,NaN,NaN,NaN,27.5
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,6.0,NaN,149.543351,0.0,NaN,NaN,NaN,NaN,NaN,0.0


## Households with exactly 1 car

In [219]:
hh_1_car.head(2)

,Name_1,Num_of_Household_with_1_Car_p75
0,ADDISON ROAD-SEAT PLEASANT,143.0
1,ANACOSTIA,49.0


In [220]:
hh_1_car['HH_1_car'] = hh_1_car['Num_of_Household_with_1_Car_p75']
hh_1_car.drop('Num_of_Household_with_1_Car_p75', axis=1, inplace=True)

In [221]:
stationlevel_75mile = stationlevel_75mile.merge(hh_1_car, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,intercityhub,prop_str_dens,Jobs_transit_p75,convenience_storecount_p75,highschoolenroll,college_enroll,college_hh,High_School_hh,HH_0_car,HH_1_car
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,NaN,485.968261,53581136.0,NaN,777.0,NaN,32.0,177.0,16.0,143.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,40970.0,383.409456,103249729.0,7.0,3634.0,NaN,NaN,NaN,8.0,49.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,NaN,933.339602,108026328.0,8.0,1616.0,232.0,NaN,NaN,10.0,15.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,27714.0,513.617499,12676878.0,1.0,NaN,NaN,NaN,NaN,27.5,113.0
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,NaN,149.543351,0.0,NaN,NaN,NaN,NaN,NaN,0.0,33.5


## Households with more than 1 car 

In [222]:
hh_morethan1_car.head(2)

,Name_1,Num_of_Household_with_More_than_One_Car_p75
0,ADDISON ROAD-SEAT PLEASANT,214.0
1,ANACOSTIA,15.0


In [223]:
hh_morethan1_car['HH_more1_car'] = hh_morethan1_car['Num_of_Household_with_More_than_One_Car_p75']
hh_morethan1_car.drop('Num_of_Household_with_More_than_One_Car_p75', axis=1, inplace=True)

In [224]:
stationlevel_75mile = stationlevel_75mile.merge(hh_morethan1_car, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,prop_str_dens,Jobs_transit_p75,convenience_storecount_p75,highschoolenroll,college_enroll,college_hh,High_School_hh,HH_0_car,HH_1_car,HH_more1_car
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,485.968261,53581136.0,NaN,777.0,NaN,32.0,177.0,16.0,143.0,214.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,383.409456,103249729.0,7.0,3634.0,NaN,NaN,NaN,8.0,49.0,15.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,933.339602,108026328.0,8.0,1616.0,232.0,NaN,NaN,10.0,15.0,0.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,513.617499,12676878.0,1.0,NaN,NaN,NaN,NaN,27.5,113.0,26.5
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,149.543351,0.0,NaN,NaN,NaN,NaN,NaN,0.0,33.5,225.0


## Public Admin jobs

In [225]:
public_admin_jobs.head(2)

,Name_1,CNS20
0,ADDISON ROAD-SEAT PLEASANT,140.917154
1,ANACOSTIA,514.938876


In [226]:
public_admin_jobs['pub_admin_jobs'] = public_admin_jobs['CNS20']
public_admin_jobs.drop('CNS20', axis=1, inplace=True)

In [227]:
stationlevel_75mile = stationlevel_75mile.merge(public_admin_jobs, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,Jobs_transit_p75,convenience_storecount_p75,highschoolenroll,college_enroll,college_hh,High_School_hh,HH_0_car,HH_1_car,HH_more1_car,pub_admin_jobs
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,53581136.0,NaN,777.0,NaN,32.0,177.0,16.0,143.0,214.0,140.917154
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,103249729.0,7.0,3634.0,NaN,NaN,NaN,8.0,49.0,15.0,514.938876
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,108026328.0,8.0,1616.0,232.0,NaN,NaN,10.0,15.0,0.0,26309.021191
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,12676878.0,1.0,NaN,NaN,NaN,NaN,27.5,113.0,26.5,16.292949
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,33.5,225.0,0.000000


## Number of Affordable Housing Units/Senior housing facilities within each walksheds

In [228]:
senior_housing_afford.head(2)

,Name_1,TotalUnits_p75
0,ADDISON ROAD-SEAT PLEASANT,469
1,ANACOSTIA,4013


In [229]:
senior_housing_afford['housing_units_afford'] = senior_housing_afford['TotalUnits_p75']
senior_housing_afford.drop('TotalUnits_p75', axis=1, inplace=True)

In [230]:
stationlevel_75mile = stationlevel_75mile.merge(senior_housing_afford, right_on = 'Name_1', left_on='Name_1', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,convenience_storecount_p75,highschoolenroll,college_enroll,college_hh,High_School_hh,HH_0_car,HH_1_car,HH_more1_car,pub_admin_jobs,housing_units_afford
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,NaN,777.0,NaN,32.0,177.0,16.0,143.0,214.0,140.917154,469.0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,7.0,3634.0,NaN,NaN,NaN,8.0,49.0,15.0,514.938876,4013.0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,8.0,1616.0,232.0,NaN,NaN,10.0,15.0,0.0,26309.021191,1111.0
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,1.0,NaN,NaN,NaN,NaN,27.5,113.0,26.5,16.292949,134.0
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,33.5,225.0,0.000000,NaN


In [231]:
stationlevel_75mile.columns

Index(['ID', 'AM_AVG_TRAINS', 'STATION_ID', 'PM_AVG_TRAINS',
       'ALLPERIODS_AVG_TRAINS', 'proportionhouses', 'Total Households',
       'PARKING_CAPACITY', 'bus_line_count', 'bus_stop_count', 'All_Jobs_p75',
       'distance_to_core', 'Name_1', 'Proportion_night_weekend_jobs_p75',
       'Total_Nine_to_Five_workers_p75', 'Proportion_education_jobs_p75',
       'Median_household_income_p75', 'MSTN', 'terminal_dummy_2023',
       'bike_cap', 'ctpp_jobs', 'HH_below_50k_p75', 'hotelcount_p75',
       'entertainmentcount_p75', 'restaurantcount_p75', 'intercityhub',
       'prop_str_dens', 'Jobs_transit_p75', 'convenience_storecount_p75',
       'highschoolenroll', 'college_enroll', 'college_hh', 'High_School_hh',
       'HH_0_car', 'HH_1_car', 'HH_more1_car', 'pub_admin_jobs',
       'housing_units_afford'],
      dtype='object')

In [232]:
stationlevel_75mile.columns

Index(['ID', 'AM_AVG_TRAINS', 'STATION_ID', 'PM_AVG_TRAINS',
       'ALLPERIODS_AVG_TRAINS', 'proportionhouses', 'Total Households',
       'PARKING_CAPACITY', 'bus_line_count', 'bus_stop_count', 'All_Jobs_p75',
       'distance_to_core', 'Name_1', 'Proportion_night_weekend_jobs_p75',
       'Total_Nine_to_Five_workers_p75', 'Proportion_education_jobs_p75',
       'Median_household_income_p75', 'MSTN', 'terminal_dummy_2023',
       'bike_cap', 'ctpp_jobs', 'HH_below_50k_p75', 'hotelcount_p75',
       'entertainmentcount_p75', 'restaurantcount_p75', 'intercityhub',
       'prop_str_dens', 'Jobs_transit_p75', 'convenience_storecount_p75',
       'highschoolenroll', 'college_enroll', 'college_hh', 'High_School_hh',
       'HH_0_car', 'HH_1_car', 'HH_more1_car', 'pub_admin_jobs',
       'housing_units_afford'],
      dtype='object')

In [233]:
stationlevel_75mile.shape

(98, 38)

## SoP and Bike Lanes

In [234]:
sop.head(2)

,ID,Name_1,bikelane,form3norm,dens2norm,prox3norm,conn6norm,parks2norm,peds4norm,safenorm,traffic5norm,aesttot3norm,paf2norm,sop7norm
0,MSTN_062,ADDISON ROAD-SEAT PLEASANT,26,83.592003,54.188693,3.348165,76.889795,1.303186,30.502679,76.009264,50.159291,52.956552,0.320397,46.884765
1,MSTN_001,ANACOSTIA,18,80.752339,40.849767,10.285137,75.577863,3.443532,31.693741,77.080763,54.616166,50.629299,0.346110,47.471791


In [235]:
stationlevel_75mile = stationlevel_75mile.merge(sop, right_on = 'ID', left_on='ID', how = 'left')
stationlevel_75mile.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,dens2norm,prox3norm,conn6norm,parks2norm,peds4norm,safenorm,traffic5norm,aesttot3norm,paf2norm,sop7norm
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,54.188693,3.348165,76.889795,1.303186,30.502679,76.009264,50.159291,52.956552,0.320397,46.884765
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,40.849767,10.285137,75.577863,3.443532,31.693741,77.080763,54.616166,50.629299,0.346110,47.471791
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,57.471003,32.165827,74.207390,44.088197,37.761560,85.092297,60.362712,59.127210,1.646596,64.598462
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,32.615821,6.793607,77.646003,13.976022,33.229987,80.842527,50.263900,51.218987,2.807484,46.922339
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,34.292517,2.666423,76.202966,0.170262,22.081078,73.977363,46.033364,54.023581,0.000008,41.704789


In [236]:
stationlevel_75mile.shape

(98, 51)

# Separate .75 and .5 Station Level Variables into Urban and Suburban

In [237]:
urban_dummy = pd.read_excel("../Data/Stations for 0.75 Mile Consideration.xlsx").drop(['PRIMARY_NAME_FY23'], axis = 1)

In [238]:
urban_dummy.head()

,ID,urban_dummy
0,MSTN_001,0
1,MSTN_002,1
2,MSTN_003,1
3,MSTN_004,1
4,MSTN_005,1


In [239]:
stationlevel_5mile

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,dens2norm,prox3norm,conn6norm,parks2norm,peds4norm,safenorm,traffic5norm,aesttot3norm,paf2norm,sop7norm
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,57.880421,4.287417,74.808946,0.000000,30.254891,82.871252,50.478578,54.865260,0.000008,48.880712
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,41.015569,11.695735,74.701052,4.361464,30.614129,77.408088,53.612582,50.533501,0.556780,47.302160
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,62.777470,39.865198,71.479741,50.846523,40.163963,85.376394,61.637340,61.840846,3.617257,69.215061
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,21.897402,8.711603,72.984694,21.256899,36.410315,80.998596,47.971883,55.101324,0.000008,48.931768
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,30.287652,1.904513,73.178281,0.000000,19.069826,74.159322,43.160322,56.933369,0.000008,40.468556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,MSTN_059,28.0,MSTN_059,26.0,2.904762,0.541509,1047.0,2009.0,15.0,9.0,...,36.143095,7.059750,72.305277,0.000000,27.317941,70.144204,46.115917,56.166206,0.639404,43.543391
94,MSTN_055,62.0,MSTN_055,56.0,6.465116,1.503493,2960.0,453.0,18.0,14.0,...,51.264546,6.610807,77.980792,25.655554,30.517402,81.029052,52.511186,50.430402,1.221443,48.908760
95,MSTN_046,56.0,MSTN_046,52.0,5.195652,2.712232,3879.0,977.0,29.0,20.0,...,39.801016,11.524703,67.840142,1.957149,33.397199,75.518159,49.499039,44.988002,0.678324,42.499103
96,MSTN_091,27.0,MSTN_091,24.0,2.717391,1.226262,3915.0,2300.0,6.0,1.0,...,36.389132,8.151990,75.507448,23.685127,30.135680,74.843689,48.066269,47.592525,5.904736,45.563617


In [240]:
stationlevel_5mile2 = stationlevel_5mile.merge(urban_dummy, on='ID', how = 'left')
stationlevel_5mile2.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,prox3norm,conn6norm,parks2norm,peds4norm,safenorm,traffic5norm,aesttot3norm,paf2norm,sop7norm,urban_dummy
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,0.528921,1663.0,1268.0,36.0,18.0,...,4.287417,74.808946,0.000000,30.254891,82.871252,50.478578,54.865260,0.000008,48.880712,0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,2.075704,3639.0,808.0,64.0,36.0,...,11.695735,74.701052,4.361464,30.614129,77.408088,53.612582,50.533501,0.556780,47.302160,0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,39.865198,71.479741,50.846523,40.163963,85.376394,61.637340,61.840846,3.617257,69.215061,1
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,8.711603,72.984694,21.256899,36.410315,80.998596,47.971883,55.101324,0.000008,48.931768,1
4,MSTN_097,NaN,NaN,NaN,NaN,0.555594,936.0,1555.0,0.0,0.0,...,1.904513,73.178281,0.000000,19.069826,74.159322,43.160322,56.933369,0.000008,40.468556,0


In [241]:
stationlevel_75mile2 = stationlevel_75mile.merge(urban_dummy, on='ID', how = 'left')
stationlevel_75mile2.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,prox3norm,conn6norm,parks2norm,peds4norm,safenorm,traffic5norm,aesttot3norm,paf2norm,sop7norm,urban_dummy
0,MSTN_062,53.0,MSTN_062,48.0,5.340909,3.042234,4450.0,1268.0,36.0,33.0,...,3.348165,76.889795,1.303186,30.502679,76.009264,50.159291,52.956552,0.320397,46.884765,0
1,MSTN_001,33.0,MSTN_001,34.0,3.400000,5.065567,6579.0,808.0,69.0,75.0,...,10.285137,75.577863,3.443532,31.693741,77.080763,54.616166,50.629299,0.346110,47.471791,0
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,9.904623,12975.0,NaN,137.0,145.0,...,32.165827,74.207390,44.088197,37.761560,85.092297,60.362712,59.127210,1.646596,64.598462,1
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,2.142590,3154.0,NaN,17.0,1.0,...,6.793607,77.646003,13.976022,33.229987,80.842527,50.263900,51.218987,2.807484,46.922339,1
4,MSTN_097,NaN,NaN,NaN,NaN,2.967461,4375.0,1555.0,0.0,0.0,...,2.666423,76.202966,0.170262,22.081078,73.977363,46.033364,54.023581,0.000008,41.704789,0


In [242]:
stationlevel_75mile2.columns = stationlevel_75mile2.columns.str.replace('_p75', '')
stationlevel_5mile2.columns = stationlevel_5mile2.columns.str.replace('_p5', '')

In [243]:
urban = stationlevel_5mile2[stationlevel_5mile2['urban_dummy'] == 1]
suburbs = stationlevel_75mile2[stationlevel_75mile2['urban_dummy'] == 0]

In [244]:
split = pd.concat([urban, suburbs])
split.shape

(98, 52)

In [245]:
split.head()

,ID,AM_AVG_TRAINS,STATION_ID,PM_AVG_TRAINS,ALLPERIODS_AVG_TRAINS,proportionhouses,Total Households,PARKING_CAPACITY,bus_line_count,bus_stop_count,...,prox3norm,conn6norm,parks2norm,peds4norm,safenorm,traffic5norm,aesttot3norm,paf2norm,sop7norm,urban_dummy
2,MSTN_002,62.0,MSTN_002,62.0,6.340909,2.214692,3007.0,NaN,86.0,50.0,...,39.865198,71.479741,50.846523,40.163963,85.376394,61.637340,61.840846,3.617257,69.215061,1
3,MSTN_065,27.0,MSTN_065,24.0,2.904762,0.000003,1.0,NaN,9.0,0.0,...,8.711603,72.984694,21.256899,36.410315,80.998596,47.971883,55.101324,0.000008,48.931768,1
5,MSTN_068,53.0,MSTN_068,50.0,5.465116,11.603907,13574.0,NaN,35.0,30.0,...,22.260210,75.345384,18.612456,37.309540,83.210485,58.343926,51.155508,7.318810,56.038042,1
6,MSTN_003,51.0,MSTN_003,48.0,5.318182,3.304806,4621.0,NaN,30.0,34.0,...,9.558899,76.637398,9.548447,33.763914,78.760511,58.799337,50.953189,0.599569,49.072915,1
7,MSTN_052,56.0,MSTN_052,52.0,5.642857,5.234589,7416.0,NaN,11.0,22.0,...,21.597336,73.177772,19.327775,35.675442,78.922981,56.792387,52.493394,6.348291,55.249112,1


# Merge Station Level Variables into OD Dataframe

In [246]:
allvars_dataframe_new.columns

Index(['ID_x', 'PRIMARY_NAME_FY23_x', 'walkshed_filename_x',
       'WMATA_filename_x', 'track_miles_names_x', 'auto_filename_x', 'ID_y',
       'PRIMARY_NAME_FY23_y', 'walkshed_filename_y', 'WMATA_filename_y',
       'track_miles_names_y', 'auto_filename_y', 'O', 'D', 'track_miles',
       'pairs', 'COMP_MILE', 'PEAK_FARE', 'OFF_PEAK_FARE', 'SD_FARE',
       'TRAVEL_TIME', 'O_MSTN_ID', 'D_MSTN_ID', 'peak_fare_per_mile2',
       'off_peak_fare_per_mile', 'am_new_auto_tt2', 'am_new_auto_tt_per_mile2',
       'pm_new_auto_tt2', 'pm_new_auto_tt_per_mile2', 'off_new_auto_tt2',
       'off_new_auto_tt_per_mile2', 'bus_tt', 'bus_tt_per_mile',
       'am_parking_user', 'pm_parking_user', 'off_parking_user',
       'bike_traveltime', 'google_driving_minutes', 'google_driving_miles',
       'bus_transit_minutes', 'metro_transit_minutes',
       'bus_competativeness_index', 'trip_time_mins'],
      dtype='object')

In [247]:
split.rename(columns = {'ID':'MSTN_ID'}, inplace = True)

In [248]:
allvars_dataframe_new2 = allvars_dataframe_new.merge(split, left_on='ID_x', right_on='MSTN_ID')
allvars_dataframe_new2 = allvars_dataframe_new2.merge(split, left_on='ID_y', right_on='MSTN_ID')
allvars_dataframe_new2.head()

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,prox3norm_y,conn6norm_y,parks2norm_y,peds4norm_y,safenorm_y,traffic5norm_y,aesttot3norm_y,paf2norm_y,sop7norm_y,urban_dummy_y
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,3.348165,76.889795,1.303186,30.502679,76.009264,50.159291,52.956552,0.320397,46.884765,0
1,MSTN_001,Anacostia,ANACOSTIA,Anacostia,Anacostia,ANACOSTIA,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,3.348165,76.889795,1.303186,30.502679,76.009264,50.159291,52.956552,0.320397,46.884765,0
2,MSTN_002,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,Archives-Navy Memorial,Archives,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,3.348165,76.889795,1.303186,30.502679,76.009264,50.159291,52.956552,0.320397,46.884765,0
3,MSTN_065,Arlington Cemetery,ARLINGTON CEMETERY,Arlington Cemetery,Arlington Cemetery,ARLINGTON CEMETERY,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,3.348165,76.889795,1.303186,30.502679,76.009264,50.159291,52.956552,0.320397,46.884765,0
4,MSTN_068,Ballston-MU,BALLSTON-MU,Ballston,Ballston-MU,BALLSTON-MU,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,3.348165,76.889795,1.303186,30.502679,76.009264,50.159291,52.956552,0.320397,46.884765,0


In [249]:
allvars_dataframe_new2.shape

(8281, 147)

# Final Touches and Exporting

* Turn column names with _x or _y to _D and _O
* Drop redundant columns
* Fill 0s and NAs with 0.01 to prevent errors with taking the natural log
* Fill terminal dummy columns with 0

In [250]:
allvars_dataframe_new2.columns = allvars_dataframe_new2.columns.str.replace('_x','_D')
allvars_dataframe_new2.columns = allvars_dataframe_new2.columns.str.replace('_y','_O')
allvars_dataframe_new2.columns

Index(['ID_D', 'PRIMARY_NAME_FY23_D', 'walkshed_filename_D',
       'WMATA_filename_D', 'track_miles_names_D', 'auto_filename_D', 'ID_O',
       'PRIMARY_NAME_FY23_O', 'walkshed_filename_O', 'WMATA_filename_O',
       ...
       'prox3norm_O', 'conn6norm_O', 'parks2norm_O', 'peds4norm_O',
       'safenorm_O', 'traffic5norm_O', 'aesttot3norm_O', 'paf2norm_O',
       'sop7norm_O', 'urban_dummy_O'],
      dtype='object', length=147)

In [251]:
allvars_dataframe_new.head(2)

,ID_x,PRIMARY_NAME_FY23_x,walkshed_filename_x,WMATA_filename_x,track_miles_names_x,auto_filename_x,ID_y,PRIMARY_NAME_FY23_y,walkshed_filename_y,WMATA_filename_y,...,am_parking_user,pm_parking_user,off_parking_user,bike_traveltime,google_driving_minutes,google_driving_miles,bus_transit_minutes,metro_transit_minutes,bus_competativeness_index,trip_time_mins
0,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.000000,NaN
1,MSTN_062,Addison Road,ADDISON ROAD-SEAT PLEASANT,Addison Road,Addison Road,ADDISON ROAD-SEAT PLEASANT,MSTN_001,Anacostia,ANACOSTIA,Anacostia,...,0.0,0.0,0.0,NaN,18.05,7.59,35.82,31,0.865438,46.482155


In [252]:
#cols_transform = allvars_dataframe_new.columns[6:]

#for col in cols_transform:
#    allvars_dataframe_new[f'log_{col}'] = np.log1p(allvars_dataframe_new[col])

In [253]:
allvars_dataframe_new2[['terminal_dummy_2023_O', 'terminal_dummy_2023_D', 'urban_dummy_O', 'urban_dummy_D']] = allvars_dataframe_new2[['terminal_dummy_2023_O', 'terminal_dummy_2023_D', 'urban_dummy_O', 'urban_dummy_D' ]].fillna(0)

In [254]:
allvars_dataframe_new2.to_csv("Outputs/allvars_dataframe_new.csv")

In [255]:
# allvars_dataframe_new2.head()